# Model Comparisons

Take the learnings and use 500 seq_len and 16 batch_size

1. LSTM
1. BiLSTM
3. LSTM + Dense
3. BiLSTM + Dense
2. 2 Hidden LSTM
2. 2 Hidden BiLSTM
2. 2 Hidden LSTM + Dense
2. 2 Hidden BiLSTM + Dense
5. Threshold

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, mean_absolute_error, confusion_matrix, f1_score, classification_report, matthews_corrcoef, recall_score
from sklearn.model_selection import train_test_split, KFold
import seaborn as sns
import matplotlib.pyplot as plt
import time
import os
import sys

import json
from pprint import pprint

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, accuracy_score


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional



In [2]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Get the current working directory (where your notebook is)
current_directory = os.getcwd()


# Construct the full path to your data_utils.py file
data_utils_path = '/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Thesis Files/data_utils.py'  # Replace with the actual path

# Add the directory containing data_utils.py to the Python path
sys.path.append(os.path.dirname(data_utils_path))  # Add parent directory of data_utils.py
# Now you can import the custom module
import data_utils as du
import analysis_utils as au

sys.path = current_directory
print(sys.path)

Mounted at /content/drive
/content


## Model Set-Up

### Simple Models for comparison

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Attention, Dropout

# 1a. LSTM
def create_lstm(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(32, return_sequences=True),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax")
    ], name = "LSTM")
    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# 1b. BiLSTM
def create_bidirectional_lstm(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax")
    ], name = "BiLSTM")
    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# 2a. LSTM + Dense
def create_lstm_with_dense(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(32, return_sequences=True),
        Dense(16, activation="relu"),  # Dense layer added
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax")
    ], name = "LSTM_Dense")
    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# 2b. BiLSTM + Dense
def create_bidirectional_lstm_with_dense(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dense(16, activation="relu"),  # Dense layer added
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax")
    ], name = "BiLSTM_Dense")
    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# 3a. 2 Hidden LSTM + Dense
def create_2_hidden_lstm_with_dense(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, return_sequences=True),  # First LSTM layer
        LSTM(32, return_sequences=True),  # Second LSTM layer
        Dense(16, activation="relu"),  # Dense layer added
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax")
    ], name = "LSTM_Deep")
    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

# 3b. 2 Hidden BiLSTM + Dense
def create_2_hidden_bidirectional_lstm_with_dense(input_shape, binary):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(LSTM(64, return_sequences=True)),  # First BiLSTM layer
        Bidirectional(LSTM(32, return_sequences=True)),  # Second BiLSTM layer
        Dense(16, activation="relu"),  # Dense layer added
        Dense(1 if binary else 5, activation="sigmoid" if binary else "softmax")
    ], name = "BiLSTM_Deep")
    loss = "binary_crossentropy" if binary else "sparse_categorical_crossentropy"
    model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
    return model

### Create Model Dicts

In [5]:
def create_models_simple(input_shape, binary):
    """
    Creates and compiles all the models, returning them in a dictionary.

    Args:
        input_shape: Tuple specifying the input shape for the models.
        binary: Boolean indicating if the problem is binary classification.

    Returns:
        A dictionary where keys are model names and values are dictionaries
        containing the model and its parameter count.
    """

    # 1. LSTM Standard
    lstm_model = create_lstm(input_shape, binary)

    # 2. BiLSTM
    bilstm_model = create_bidirectional_lstm(input_shape, binary)

    # 3. LSTM + Dense
    lstm_dense_model = create_lstm_with_dense(input_shape, binary)

    # 4. BiLSTM + Dense
    bilstm_dense_model = create_bidirectional_lstm_with_dense(input_shape, binary)

    # 5. 2 Hidden LSTM + Dense
    lstm_2_hidden_dense_model = create_2_hidden_lstm_with_dense(input_shape, binary)

    # 6. 2 Hidden BiLSTM + Dense
    bilstm_2_hidden_dense_model = create_2_hidden_bidirectional_lstm_with_dense(input_shape, binary)

    models = {
        "LSTM": {"model": lstm_model, "parameters": lstm_model.count_params()},
        "LSTM_Dense": {"model": lstm_dense_model, "parameters": lstm_dense_model.count_params()},
        "LSTM_Deep": {"model": lstm_2_hidden_dense_model, "parameters": lstm_2_hidden_dense_model.count_params()},
        "BiLSTM": {"model": bilstm_model, "parameters": bilstm_model.count_params()},
        "BiLSTM_Dense": {"model": bilstm_dense_model, "parameters": bilstm_dense_model.count_params()},
        "BiLSTM_Deep": {"model": bilstm_2_hidden_dense_model, "parameters": bilstm_2_hidden_dense_model.count_params()},
    }

    return models

In [6]:
models_dict = create_models_simple((500, 1), True)
print(models_dict)
models_dict = create_models_simple((500, 5), True)
print(models_dict)
models_dict = create_models_simple((500, 25), True)
print(models_dict)

{'LSTM': {'model': <Sequential name=LSTM, built=True>, 'parameters': 4385}, 'LSTM_Dense': {'model': <Sequential name=LSTM_Dense, built=True>, 'parameters': 4897}, 'LSTM_Deep': {'model': <Sequential name=LSTM_Deep, built=True>, 'parameters': 29857}, 'BiLSTM': {'model': <Sequential name=BiLSTM, built=True>, 'parameters': 8769}, 'BiLSTM_Dense': {'model': <Sequential name=BiLSTM_Dense, built=True>, 'parameters': 9761}, 'BiLSTM_Deep': {'model': <Sequential name=BiLSTM_Deep, built=True>, 'parameters': 76065}}
{'LSTM': {'model': <Sequential name=LSTM, built=True>, 'parameters': 4897}, 'LSTM_Dense': {'model': <Sequential name=LSTM_Dense, built=True>, 'parameters': 5409}, 'LSTM_Deep': {'model': <Sequential name=LSTM_Deep, built=True>, 'parameters': 30881}, 'BiLSTM': {'model': <Sequential name=BiLSTM, built=True>, 'parameters': 9793}, 'BiLSTM_Dense': {'model': <Sequential name=BiLSTM_Dense, built=True>, 'parameters': 10785}, 'BiLSTM_Deep': {'model': <Sequential name=BiLSTM_Deep, built=True>, 'pa

In [7]:
def model_funcs_simple():
    """
    Creates a dictionary of model creation functions.

    Args:
        input_shape: Tuple specifying the input shape for the models.
        binary: Boolean indicating if the problem is binary classification.

    Returns:
        A dictionary where keys are model names and values are the
        corresponding model creation functions.
    """
    models = {
        "LSTM": create_lstm,
        "LSTM_Dense": create_lstm_with_dense,
        "LSTM_Deep": create_2_hidden_lstm_with_dense,
        "BiLSTM": create_bidirectional_lstm,
        "BiLSTM_Dense": create_bidirectional_lstm_with_dense,
        "BiLSTM_Deep": create_2_hidden_bidirectional_lstm_with_dense,
    }

    return models

Threshold Model

In [8]:
import numpy as np

def compute_thresholds(X, std_factor=1.0):
    """
    Computes dynamic thresholds using mean + std dev, ensuring no negative thresholds.

    :param X: Training data of shape (n_samples, seq_len, num_features)
    :param std_factor: Multiplier for standard deviation (e.g., 1.0 means mean + 1 std dev)
    :return: Computed thresholds of shape (1, 1, num_features)
    """
    feature_data = X.reshape(-1, X.shape[-1])  # Flatten across samples & time: (n_samples * seq_len, num_features)
    mean_values = np.mean(feature_data, axis=0)
    std_values = np.std(feature_data, axis=0)

    # Compute thresholds (only allow positive values)
    thresholds = np.maximum(mean_values + std_factor * std_values, 0)

    # Reshape to broadcast properly: (1, 1, num_features)
    return thresholds.reshape(1, 1, -1)

def predict_binary(X, thresholds):
    """
    Binary classification: Predict whether each timestep contains a burst (1) or not (0).

    :param X: Input data of shape (n_samples, seq_len, num_features)
    :param thresholds: Thresholds of shape (1, 1, num_features)
    :return: Binary predictions of shape (n_samples, seq_len)
    """
    # Compare with thresholds for each feature, then check if any feature exceeds its threshold
    return (X > thresholds).any(axis=-1).astype(int)

def predict_multiclass(X, thresholds):
    """
    Multiclass classification: Count how many features exceed their threshold per timestep.

    :param X: Input data of shape (n_samples, seq_len, num_features)
    :param thresholds: Thresholds of shape (1, 1, num_features)
    :return: Multiclass predictions of shape (n_samples, seq_len), values 0-4
    """
    return np.sum(X > thresholds, axis=-1)

# Example Usage
np.random.seed(42)
X_train = np.random.rand(100, 500, 4)  # Fake training data (100 samples, 500 timesteps, 4 features)
X_test = np.random.rand(20, 500, 4)    # Fake test data (20 samples, 500 timesteps, 4 features)

# Compute dynamic thresholds from training data
thresholds = compute_thresholds(X_train, std_factor=1.5)

# Predict bursts using binary and multiclass models
binary_preds = predict_binary(X_test, thresholds)
multiclass_preds = predict_multiclass(X_test, thresholds)

print(f"Binary Predictions Shape: {binary_preds.shape} (Should be (20, 500))")
print(f"Multiclass Predictions Shape: {multiclass_preds.shape} (Should be (20, 500))")


Binary Predictions Shape: (20, 500) (Should be (20, 500))
Multiclass Predictions Shape: (20, 500) (Should be (20, 500))


## Data Import

In [9]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Including Features/5_class_mid_noise_30s_features_vec.npy"
label_path = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Including Features/5_class_mid_noise_30s_numeric_label_vec.npy"

data_vec, label_vec = du.load_data(data_path, label_path)

du.data_info(data_vec)

DATASET INFO:
Shape: (10, 20, 5, 7500, 25)

No of Samples: 10

No of Frequencies: 20
Freqency values: [np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(11), np.int64(13), np.int64(16), np.int64(18), np.int64(22), np.int64(26), np.int64(31), np.int64(36), np.int64(43), np.int64(51), np.int64(60), np.int64(71), np.int64(84), np.int64(100)]

No of noise ratios: 5
Signal to Noise ratios (in db) [np.int64(-10), np.int64(-8), np.int64(-6), np.int64(-4), np.int64(-2), np.int64(0), np.int64(2), np.int64(4), np.int64(6), np.int64(8)]

No of Datapoints: 7500

No of Features per Datapoint: 25 (signal, hilbert amp, 20 wavelets for each freq)


In [10]:
data_vec_trim = data_vec[:, :, :]
label_vec_trim = label_vec[:, :, :]

print(data_vec_trim.shape)
print(label_vec_trim.shape)

(10, 20, 5, 7500, 25)
(10, 20, 5, 7500)


In [11]:
## Class distribution
unique, counts = np.unique(label_vec_trim.flatten(), return_counts=True)
print(f"Class distribution: {dict(zip(unique, counts))}")
print(f"Noise  Percentage: {(counts[0]/(sum(counts))):.4f}")
print(f"Theta Percentage: {(counts[1]/(sum(counts))):.4f}")
print(f"Alpha Percentage: {(counts[2]/(sum(counts))):.4f}")
print(f"Beta Percentage: {(counts[3]/(sum(counts))):.4f}")
print(f"Gamma Percentage: {(counts[4]/(sum(counts))):.4f}")

Class distribution: {np.int64(0): np.int64(4008043), np.int64(1): np.int64(882021), np.int64(2): np.int64(882177), np.int64(3): np.int64(886981), np.int64(4): np.int64(840778)}
Noise  Percentage: 0.5344
Theta Percentage: 0.1176
Alpha Percentage: 0.1176
Beta Percentage: 0.1183
Gamma Percentage: 0.1121


In [12]:
def preprocess_data(data_vector, label_vector):
    """
    Preprocesses data and label vectors to the desired shape (S, timesteps, I).
    Handles various input shapes, including timesteps from 1000 and up:
    - S: N_of_samples
    - F: N_of_frequencies
    - N: N_of_noise_burst_ratios
    - I: N_of_Input_Features
    - 2D: (S, timesteps)
    - 3D: (S, F, timesteps) or (S, timesteps, I)
    - 4D: (S, F, N, timesteps) or (S, F, timesteps, I)
    - 5D: (S, F, N, timesteps, I)

    Args:
        data_vector (np.ndarray): The input data vector.
        label_vector (np.ndarray): The input label vector.

    Returns:
        tuple: A tuple containing the preprocessed data and label vectors.
    """

    def reshape_vector(vector):
        """Helper function to reshape a single vector."""
        if vector.ndim == 2:  # (S, timesteps)
            # Reshape 2D to 3D by adding a feature dimension with I = 1
            vector = vector.reshape(vector.shape[0], vector.shape[1], 1)
        elif vector.ndim == 3:
            if vector.shape[1] < 1000 and vector.shape[2] >= 1000:  # (S, F, timesteps)
                # Assume timesteps is the last dimension, move it to the second position
                vector = vector.transpose(0, 2, 1)
            # else: (S, timesteps, I) - already in desired format
        elif vector.ndim == 4:
            if vector.shape[2] < 1000 and vector.shape[3] >= 1000:  # (S, F, N, timesteps)
                # Reshape by flattening the first three dimensions
                N = vector.shape[0] * vector.shape[1] * vector.shape[2]
                vector = vector.reshape(N, vector.shape[3], 1)
            # Assume its (S, F, timesteps, I), flatten first 2 to get (N, timesteps, I)
            elif vector.shape[2] >= 1000 and vector.shape[3] < 1000:
                N = vector.shape[0] * vector.shape[1]
                vector = vector.reshape(N, vector.shape[2], vector.shape[3])
            else:
                raise ValueError("4D input shape not recognized. Please check the dimensions.")
        elif vector.ndim == 5:  # (S, F, N, timesteps, I)
            # Reshape by flattening the first three dimensions
            N = vector.shape[0] * vector.shape[1] * vector.shape[2]
            vector = vector.reshape(N, vector.shape[3], vector.shape[4])
        else:
            raise ValueError("Input vector must be 2D, 3D, 4D, or 5D.")
        return vector


    # Reshape data and label vectors independently
    data_vector = reshape_vector(data_vector)
    label_vector = reshape_vector(label_vector)

    return data_vector, label_vector

In [13]:
import numpy as np

def split_sequences_keep_data(X, y, new_seq_len):
    """
    Splits input sequences and their per-timestep labels into smaller sequences of length `new_seq_len`.

    Args:
        X (numpy array): Input data of shape (samples, seq_len, features).
        y (numpy array): Labels of shape (samples, seq_len) or (samples, seq_len, ...).
        new_seq_len (int): Desired sequence length.

    Returns:
        X_new: New input data with shape (new_samples, new_seq_len, features).
        y_new: New labels with shape (new_samples, new_seq_len, ...).
    """
    num_samples, original_seq_len, num_features = X.shape
    if original_seq_len % new_seq_len != 0:
        print(f"Warning: Original seq_len ({original_seq_len}) is not a multiple of new_seq_len ({new_seq_len}).")

    # Number of new samples per original sequence
    num_segments = original_seq_len // new_seq_len

    # Create new datasets by reshaping
    X_new = np.reshape(X[:, :num_segments * new_seq_len, :],
                       (-1, new_seq_len, num_features))  # Shape: (samples * num_segments, new_seq_len, features)
    y_new = np.reshape(y[:, :num_segments * new_seq_len, ...],
                       (-1, new_seq_len, *y.shape[2:]))  # Adjust labels to match input shape

    return X_new, y_new


In [14]:
data_vec_shaped, label_vec_shaped = preprocess_data(data_vector=data_vec_trim, label_vector=label_vec_trim)

print(data_vec_shaped.shape)
print(label_vec_shaped.shape)

(1000, 7500, 25)
(1000, 7500, 1)


In [15]:
new_seq_len = 500

sample_increase_factor = int(data_vec_shaped.shape[1] / new_seq_len)

data_vec_reshaped, label_vec_reshaped = split_sequences_keep_data(data_vec_shaped, label_vec_shaped, new_seq_len=new_seq_len)

# Extract noise levels
noise_levels = np.arange(data_vec_trim.shape[2])  # Assuming noise levels are 0, 1, 2, 3, 4
noise_levels = np.repeat(noise_levels, data_vec_trim.shape[0] * data_vec_trim.shape[1] * sample_increase_factor)

In [16]:

print(data_vec_reshaped.shape)
print(label_vec_reshaped.shape)
print(noise_levels.shape)

(15000, 500, 25)
(15000, 500, 1)
(15000,)


In [17]:
# For testing make training data very small
# data_vec_reshaped = data_vec_reshaped[:50]
# label_vec_reshaped = label_vec_reshaped[:50]

In [18]:
# label_one_hot = []
# for label in label_vec_reshaped:
#     label_one_hot.append(du.one_hot_encode_single_sample(label, [0,1,2,3,4]))

# label_one_hot = np.array(label_one_hot)
# label_one_hot.shape

## Data Splits

Be careful about choosing binary or not in the TT Splits

In [19]:
# Assuming `X` is your input data and `y` is your labels
test_size = 0.2  # 20% test set

hilbert_data_vec = data_vec_reshaped[:,:,:5]
signal_data_vec = data_vec_reshaped[:,:, :1]
all_data_vec = data_vec_reshaped

label_multi = label_vec_reshaped
label_binary = label_vec_reshaped.copy()
label_binary[label_binary >= 1] = 1  # Convert to binary labels

# First, split into training+validation and test sets
X_train, X_test, y_train, y_test = train_test_split(hilbert_data_vec, label_multi, test_size=test_size, random_state=42)


# Check the shapes
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Training set shape: (12000, 500, 5) (12000, 500, 1)
Test set shape: (3000, 500, 5) (3000, 500, 1)


In [20]:
models_dict = create_models_simple((1500, 5), binary=False)
models_dict

{'LSTM': {'model': <Sequential name=LSTM, built=True>, 'parameters': 5029},
 'LSTM_Dense': {'model': <Sequential name=LSTM_Dense, built=True>,
  'parameters': 5477},
 'LSTM_Deep': {'model': <Sequential name=LSTM_Deep, built=True>,
  'parameters': 30949},
 'BiLSTM': {'model': <Sequential name=BiLSTM, built=True>,
  'parameters': 10053},
 'BiLSTM_Dense': {'model': <Sequential name=BiLSTM_Dense, built=True>,
  'parameters': 10853},
 'BiLSTM_Deep': {'model': <Sequential name=BiLSTM_Deep, built=True>,
  'parameters': 78181}}

## Train Models (deprecated)

In [19]:
def evaluate_model(model, X_train, y_train, X_val, y_val, binary):
    """
    Trains and evaluates the model, returning the metrics and evaluation data.
    """


    model.summary()

    start_time = time.time()

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=1, validation_data=(X_val, y_val))

    # Measure training time
    training_time = time.time() - start_time

    start_time = time.time()

    # Evaluate
    y_pred = model.predict(X_val).argmax(axis=-1) if not binary else (model.predict(X_val) > 0.5).astype(int)
    inference_time = time.time() - start_time

    y_pred = y_pred.flatten()
    y_val = y_val.flatten()


    accuracy = accuracy_score(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    recall = recall_score(y_val, y_pred, average="binary" if binary else "macro")
    f1 = f1_score(y_val, y_pred, average="binary" if binary else "macro")
    mcc = matthews_corrcoef(y_val, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"MAE: {mae}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"MCC: {mcc}")

    return model , {
        "accuracy": accuracy,
        "mcc": mcc,
        "mae": mae,
        "recall": recall,
        "f1": f1,
        "training_time": training_time,
        "inference_time": inference_time

    }


In [20]:
def train_and_evaluate_models(models, X_train, y_train, X_val, y_val, binary):
    """
    Trains and evaluates all models in the dictionary, storing performance metrics.

    Args:
        models: Dictionary containing models with subkeys "model" and "parameters".
        X_train: Training data.
        y_train: Training labels.
        X_val: Validation data.
        y_val: Validation labels.

    Returns:
        The updated models dictionary with a new "performances" subkey for each model.
    """

    for model_name, model_data in models.items():#
        print(f"Training and evaluating {model_name}...")  # Optional: Print progress
        model = model_data["model"]  # Get the model object

        trained_model, performance_metrics = evaluate_model(model, X_train, y_train, X_val, y_val, binary)

        # Save the trained model in Dict
        model_data["model"] = trained_model

        # Store the performance metrics in the "performances" subkey
        model_data["performances"] = performance_metrics

    return models  # Return the updated models dictionary

In [21]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(12000, 500, 5)
(12000, 500, 1)
(3000, 500, 5)
(3000, 500, 1)


In [22]:
# Using Only the signal as an input feature

signal_only_models = train_and_evaluate_models(models_dict, X_train, y_train, X_test, y_test, binary=False)

Training and evaluating LSTM...


Model: "LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_24 (LSTM)                  │ (None, 1500, 32)       │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 1500, 5)        │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,029 (19.64 KB)

 Trainable params: 5,029 (19.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.4683 - loss: 1.2962 - val_accuracy: 0.7341 - val_loss: 0.7439
Epoch 2/10


KeyboardInterrupt: 

In [ ]:
signal_only_models


{'LSTM': {'model': <keras.engine.sequential.Sequential at 0x78eb28a7b850>,
  'parameters': 4385,
  'performances': {'accuracy': 0.8542,
   'mcc': 0.70743184074071,
   'mae': 0.1458,
   'recall': 0.8183711048158641,
   'f1': 0.8408574921411107,
   'confusion_matrix': array([[70353,  9047],
          [12823, 57777]]),
   'training_time': 144.01327061653137,
   'inference_time': 0.8876395225524902}},
 'BiLSTM': {'model': <keras.engine.sequential.Sequential at 0x78eb2929d150>,
  'parameters': 5041,
  'performances': {'accuracy': 0.8995066666666667,
   'mcc': 0.7982790522855711,
   'mae': 0.10049333333333334,
   'recall': 0.8911756373937677,
   'f1': 0.8930223975927556,
   'confusion_matrix': array([[72009,  7391],
          [ 7683, 62917]]),
   'training_time': 153.8624861240387,
   'inference_time': 1.3268213272094727}},
 'RNN': {'model': <keras.engine.sequential.Sequential at 0x78eb2b4e1090>,
  'parameters': 4289,
  'performances': {'accuracy': 0.83588,
   'mcc': 0.6710292799915758,
   '

In [ ]:
from IPython.display import HTML

def display_models_with_performances(models):
    """
    Displays models with existing 'performances' in an HTML table.

    Args:
        models: A dictionary where keys are model names and values are dictionaries
               containing model information, including 'performances' if available.
    """

    html = """
    <style>
    table {
        border-collapse: collapse;
        width: 100%;
    }
    th, td {
        text-align: center;
        padding: 8px;
        border: 1px solid #ddd;
    }
    th {
        background-color: #4f50b3;
    }
    </style>
    <table><thead><tr><th>Model</th><th>Parameters</th><th>Accuracy</th><th>MCC</th><th>Recall</th><th>F1</th><th>Training Time</th><th>Inference Time</th></tr></thead><tbody>"""

    for model_name, model_data in models.items():
        if "performances" in model_data:
            performances = model_data["performances"]
            html += f"<tr><td>{model_name}</td>"
            html += f"<td>{model_data['parameters']}</td>"
            html += f"<td>{performances['accuracy']:.3f}</td>"
            html += f"<td>{performances['mcc']:.3f}</td>"
            html += f"<td>{performances['recall']:.3f}</td>"
            html += f"<td>{performances['f1']:.3f}</td>"
            html += f"<td>{performances['training_time']:.1f}</td>"
            html += f"<td>{performances['inference_time']:.3f}</td></tr>"

    html += "</tbody></table>"
    display(HTML(html))

display_models_with_performances(signal_only_models)

Model,Parameters,Accuracy,MCC,Recall,F1,Training Time,Inference Time
LSTM,4385,0.854,0.707,0.818,0.841,144.0,0.888
BiLSTM,5041,0.900,0.798,0.891,0.893,153.9,1.327
RNN,4289,0.836,0.671,0.787,0.819,61.2,0.493
GRU,3393,0.838,0.674,0.816,0.825,143.5,1.025
TCN,5601,0.842,0.683,0.826,0.831,43.5,0.582
FNN,2241,0.688,0.375,0.549,0.623,11.1,0.186


## K-Fold Analysis

- Stratified k-fold
- Make function that takes in only the model_dict and returns result dict
- Create Heatlike result visualizations

### Set-Up

In [21]:
# Setting Up the different model dicts for different configs

models_siganl_beta_binary = create_models_simple((data_vec_reshaped.shape[1], 1), True)
models_singal_multi =       create_models_simple((data_vec_reshaped.shape[1], 1), False)

models_hilbert_beta_binary = create_models_simple((data_vec_reshaped.shape[1], 2), True)
models_hilbert_multi =       create_models_simple((data_vec_reshaped.shape[1], 5), False)

models_wavelet_beta_binary = create_models_simple((data_vec_reshaped.shape[1], 6), True)
models_wavelet_multi =       create_models_simple((data_vec_reshaped.shape[1], 21), False)

models_all_features_beta_binary = create_models_simple((data_vec_reshaped.shape[1], 7), True)
models_all_features_multi =       create_models_simple((data_vec_reshaped.shape[1], 25), False)


In [22]:
models_wavelet_multi

{'LSTM': {'model': <Sequential name=LSTM, built=True>, 'parameters': 7077},
 'LSTM_Dense': {'model': <Sequential name=LSTM_Dense, built=True>,
  'parameters': 7525},
 'LSTM_Deep': {'model': <Sequential name=LSTM_Deep, built=True>,
  'parameters': 35045},
 'BiLSTM': {'model': <Sequential name=BiLSTM, built=True>,
  'parameters': 14149},
 'BiLSTM_Dense': {'model': <Sequential name=BiLSTM_Dense, built=True>,
  'parameters': 14949},
 'BiLSTM_Deep': {'model': <Sequential name=BiLSTM_Deep, built=True>,
  'parameters': 86373}}

In [23]:

def stratified_kfold_split(X, y, num_folds=5):
    """
    Performs Stratified K-Fold cross-validation on sequence data.

    Args:
        X: ndarray of shape (samples, timesteps, features) -> (7500, 500, ?)
        y: ndarray of shape (samples, timesteps, 1) -> (7500, 500, 1)
        num_folds: Number of folds for cross-validation.

    Returns:
        List of (train_idx, val_idx) tuples.
    """
    # Convert y from (7500, 500, 1) → (7500, 500)
    y_flat = y[:, :, 0]  # Remove last dimension

    # Get **max class per sequence**
    y_max = np.max(y_flat, axis=1)  # Shape → (7500,)

    # Stratified K-Fold splitting
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    folds = list(skf.split(X, y_max))  # Get (train_idx, val_idx) splits

    return folds


In [24]:
from collections import defaultdict

# Train and Evaluate Function
def train_and_evaluate(models, X, y, k_folds=5, batch_size=16, epochs=10,
                       basePath="/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/LSTM Models/",
                       dir_name="signal_binary"):

    skf_folds = stratified_kfold_split(X,y,num_folds=k_folds)

    # Determines whether we work with binary data or multi data
    binary = len(np.unique(y)) == 2  # Check if only 2 unique values in y

    model_results = {model_name: {} for model_name in models.keys()}
    trained_models = {model_name: [] for model_name in models.keys()}

    for model_name, model_fn in models.items():
        acc_scores, mcc_scores, train_times, infer_times = [], [], [], []

        # *** Initialize a list to store history for each fold ***
        val_mcc = []
        val_accuracy = []
        val_losses = []
        training_losses = []
        training_accuracy = []


        for fold, (train_idx, val_idx) in enumerate(skf_folds):

            print(f"Training Model: {model_name}, Fold: {fold+1}")

            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]



            model = model_fn(input_shape=X_train.shape[1:], binary=binary)

            mcc_callback = MCCCallback(validation_data=(X_val, y_val), is_binary=binary)

            if model_name == "CNN":
                train_gen_100 = SlidingWindowGenerator(X_train, y_train, window_size=100, batch_size=batch_size*32, shuffle=False)
                val_gen_100 = SlidingWindowGenerator(X_val, y_val, window_size=100, batch_size=batch_size*32, shuffle=False)

                # start_train = time.time()
                # history = model.fit(train_gen_100, epochs=epochs, batch_size=batch_size*32, validation_data=val_gen_100, verbose=1)
                # train_time = time.time() - start_train

                # start_time = time.time()
                # y_true = []
                # y_pred = []
                # for i in range(len(val_gen_100)):
                #     X_batch, y_batch = val_gen_100[i]  # Get a batch of data
                #     y_true.extend(y_batch)
                #     y_pred_batch = model.predict(X_batch).argmax(axis=-1) if not binary else (model.predict(X_batch) > 0.5).astype(int)
                #     y_pred.extend(y_pred_batch)
                # inference_time = time.time() - start_time

            else:
                start_train = time.time()
                history = model.fit(X_train, y_train, epochs=epochs,
                                    batch_size=batch_size, validation_data=(X_val, y_val),
                                    verbose=1, callbacks=[mcc_callback])
                train_time = time.time() - start_train

                val_mccs = history.history['mcc']  # Get the validation MCC values
                training_accs = history.history['accuracy']
                val_accs = history.history['val_accuracy']
                val_loss = history.history['val_loss']
                training_loss = history.history['loss']


                start_infer = time.time()
                y_pred_probs = model.predict(X_val)
                infer_time = (time.time() - start_infer) / len(X_val)

                y_pred = model.predict(X_val).argmax(axis=-1) if not binary else (model.predict(X_val) > 0.5).astype(int)
                y_true_flat = y_val.flatten()
                y_pred_flat = y_pred.flatten()

            acc_scores.append(accuracy_score(y_true_flat, y_pred_flat))
            mcc_scores.append(matthews_corrcoef(y_true_flat, y_pred_flat))
            train_times.append(train_time)
            infer_times.append(infer_time)

            val_mcc.append(val_mccs)
            val_accuracy.append(val_accs)
            val_losses.append(val_loss)
            training_accuracy.append(training_accs)
            training_losses.append(training_loss)


            num_params = model.count_params()

            model_save_path = f"{basePath}/{dir_name}/{model_name}/{model_name}_fold_{fold+1}.h5"
            model.save(model_save_path)
            trained_models[model_name].append(model_save_path)

        model_results[model_name] = {
            "Accuracy": {"mean": np.mean(acc_scores), "std": np.std(acc_scores), "min": np.min(acc_scores), "max": np.max(acc_scores)},
            "MCC": {"mean": np.mean(mcc_scores), "std": np.std(mcc_scores), "min": np.min(mcc_scores), "max": np.max(mcc_scores)},
            "Train Time (s)": {"mean": np.mean(train_times), "std": np.std(train_times), "min": np.min(train_times), "max": np.max(train_times)},
            "Inference Time (s/sample)": {"mean": np.mean(infer_times), "std": np.std(infer_times), "min": np.min(infer_times), "max": np.max(infer_times)},
            "Training Accuracy": training_accuracy,
            "Training Loss": training_losses,
            "Validation MCC": val_mcc,
            "Validation Accuracy": val_accuracy,
            "Validation Loss": val_losses,
            "Parameters": num_params
        }

        pprint(model_results[model_name], width=1)

        final_model_save_path = f"{basePath}/{dir_name}/{model_name}/{model_name}_final.h5"
        model.save(final_model_save_path)

    return model_results, trained_models



In [25]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import matthews_corrcoef

class MCCCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, is_binary=True):
        super().__init__()
        self.validation_data = validation_data  # Tuple (x_val, y_val)
        self.is_binary = is_binary

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        x_val, y_val = self.validation_data
        y_pred = self.model.predict(x_val, verbose=0)

        # Ensure y_val is flattened (shape: (N,))
        y_true = y_val.flatten()

        if self.is_binary:
            # Binary classification: round predictions
            y_pred = np.round(y_pred).flatten()
        else:
            print(y_pred.shape)
            # Multiclass classification: use argmax (since predictions are (N, 5) softmax outputs)
            y_pred = np.argmax(y_pred, axis=-1).flatten()

        # Compute MCC
        mcc = matthews_corrcoef(y_true, y_pred)

        # Log MCC
        logs["mcc"] = mcc
        print(f"\nEpoch {epoch + 1} - MCC: {mcc:.4f}")


In [26]:
simple_models_dict = model_funcs_simple()

In [27]:
simple_models_dict

{'LSTM': <function __main__.create_lstm(input_shape, binary)>,
 'LSTM_Dense': <function __main__.create_lstm_with_dense(input_shape, binary)>,
 'LSTM_Deep': <function __main__.create_2_hidden_lstm_with_dense(input_shape, binary)>,
 'BiLSTM': <function __main__.create_bidirectional_lstm(input_shape, binary)>,
 'BiLSTM_Dense': <function __main__.create_bidirectional_lstm_with_dense(input_shape, binary)>,
 'BiLSTM_Deep': <function __main__.create_2_hidden_bidirectional_lstm_with_dense(input_shape, binary)>}

## Multi Models

In [28]:
wavelet_remove_index = [1,2,3,4]
all_indices = np.arange(data_vec_reshaped.shape[-1])
indices_to_keep = np.delete(all_indices, wavelet_remove_index)

hilbert_data_vec = data_vec_reshaped[:,:,:5]
signal_data_vec = data_vec_reshaped[:,:, :1]
wavelet_data_vec = data_vec_reshaped[:,:, indices_to_keep]
all_data_vec = data_vec_reshaped

label_multi = label_vec_reshaped
label_binary = label_vec_reshaped.copy()
label_binary[label_binary >= 1] = 1  # Convert to binary labels


In [29]:
print(signal_data_vec.shape)
print(hilbert_data_vec.shape)
print(wavelet_data_vec.shape)
print(all_data_vec.shape)
print(label_multi.shape)
print(label_binary.shape)

(15000, 500, 1)
(15000, 500, 5)
(15000, 500, 21)
(15000, 500, 25)
(15000, 500, 1)
(15000, 500, 1)


In [30]:
n_epochs = 20

All Features

Training Model: BiLSTM_Deep, Fold: 1
Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.5312 - loss: 1.2774(200, 1500, 5)

Epoch 1 - MCC: 0.6579
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 175ms/step - accuracy: 0.5333 - loss: 1.2729 - val_accuracy: 0.7850 - val_loss: 0.6623 - mcc: 0.6579
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.7942 - loss: 0.6200(200, 1500, 5)

Epoch 2 - MCC: 0.7697
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 125ms/step - accuracy: 0.7947 - loss: 0.6188 - val_accuracy: 0.8530 - val_loss: 0.4381 - mcc: 0.7697
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.8458 - loss: 0.4552(200, 1500, 5)

Epoch 3 - MCC: 0.7803
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 125ms/step - accuracy: 0.8458 - loss: 0.4550 - val_accuracy: 0.8597 - val_loss: 0.4004 - mcc: 0.7803
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.8525 - loss: 0.4256(200, 1500, 5)

Epoch 4 - MCC: 0.7908
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 134ms/step - accuracy: 0.8527 - loss: 0.4252 - 

Training Model: BiLSTM_Deep, Fold: 2
Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.4892 - loss: 1.3266(200, 1500, 5)

Epoch 1 - MCC: 0.6045
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 157ms/step - accuracy: 0.4915 - loss: 1.3217 - val_accuracy: 0.7542 - val_loss: 0.6656 - mcc: 0.6045
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.7578 - loss: 0.6563(200, 1500, 5)

Epoch 2 - MCC: 0.7772
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 143ms/step - accuracy: 0.7584 - loss: 0.6550 - val_accuracy: 0.8561 - val_loss: 0.4267 - mcc: 0.7772
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.8381 - loss: 0.4807(200, 1500, 5)

Epoch 3 - MCC: 0.8002
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.8382 - loss: 0.4803 - val_accuracy: 0.8721 - val_loss: 0.3679 - mcc: 0.8002
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.8653 - loss: 0.3908(200, 1500, 5)

Epoch 4 - MCC: 0.8223
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 144ms/step - accuracy: 0.8653 - loss: 0.3910 - v

KeyboardInterrupt: 

In [60]:
basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/LSTM Models/New"

model_results, trained_models = train_and_evaluate(simple_models_dict, X=all_data_vec, y=label_multi, epochs=n_epochs, dir_name="allF_multi", basePath=basePath)

filePath = f"{basePath}20_Epoch_AllF_Multi_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(model_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7345 - loss: 0.7342(3000, 500, 5)

Epoch 1 - MCC: 0.7823
750/750 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.7347 - loss: 0.7337 - val_accuracy: 0.8574 - val_loss: 0.3807 - mcc: 0.7823
Epoch 2/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8545 - loss: 0.3916(3000, 500, 5)

Epoch 2 - MCC: 0.8018
750/750 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - accuracy: 0.8545 - loss: 0.3915 - val_accuracy: 0.8709 - val_loss: 0.3484 - mcc: 0.8018
Epoch 3/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8716 - loss: 0.3431(3000, 500, 5)

Epoch 3 - MCC: 0.8180
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8716 - loss: 0.3431 - val_accuracy: 0.8810 - val_loss: 0.3107 - mcc: 0.8180
Epoch 4/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8815 - loss: 0.3090(3000, 500, 5)

Epoch 4 - MCC: 0.8274
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.8816 - loss: 0.309

Training Model: LSTM, Fold: 2
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7196 - loss: 0.7531(3000, 500, 5)

Epoch 1 - MCC: 0.7834
750/750 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - accuracy: 0.7198 - loss: 0.7528 - val_accuracy: 0.8590 - val_loss: 0.3755 - mcc: 0.7834
Epoch 2/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8594 - loss: 0.3747(3000, 500, 5)

Epoch 2 - MCC: 0.8056
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.8594 - loss: 0.3747 - val_accuracy: 0.8730 - val_loss: 0.3390 - mcc: 0.8056
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8722 - loss: 0.3330(3000, 500, 5)

Epoch 3 - MCC: 0.7541
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.8722 - loss: 0.3330 - val_accuracy: 0.8407 - val_loss: 0.4334 - mcc: 0.7541
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8729 - loss: 0.3360(3000, 500, 5)

Epoch 4 - MCC: 0.8247
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.8730 - loss: 0.336

Training Model: LSTM, Fold: 3
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7170 - loss: 0.7694(3000, 500, 5)

Epoch 1 - MCC: 0.7858
750/750 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.7172 - loss: 0.7688 - val_accuracy: 0.8609 - val_loss: 0.3762 - mcc: 0.7858
Epoch 2/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8657 - loss: 0.3621(3000, 500, 5)

Epoch 2 - MCC: 0.8074
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8657 - loss: 0.3621 - val_accuracy: 0.8735 - val_loss: 0.3342 - mcc: 0.8074
Epoch 3/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8714 - loss: 0.3468(3000, 500, 5)

Epoch 3 - MCC: 0.8148
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8714 - loss: 0.3468 - val_accuracy: 0.8796 - val_loss: 0.3188 - mcc: 0.8148
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8792 - loss: 0.3224(3000, 500, 5)

Epoch 4 - MCC: 0.8164
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.8792 - loss: 0.322

Training Model: LSTM, Fold: 4
Epoch 1/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7013 - loss: 0.7909(3000, 500, 5)

Epoch 1 - MCC: 0.7867
750/750 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.7017 - loss: 0.7899 - val_accuracy: 0.8607 - val_loss: 0.3806 - mcc: 0.7867
Epoch 2/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8666 - loss: 0.3614(3000, 500, 5)

Epoch 2 - MCC: 0.8073
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.8666 - loss: 0.3614 - val_accuracy: 0.8756 - val_loss: 0.3322 - mcc: 0.8073
Epoch 3/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8773 - loss: 0.3283(3000, 500, 5)

Epoch 3 - MCC: 0.8148
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.8773 - loss: 0.3283 - val_accuracy: 0.8802 - val_loss: 0.3216 - mcc: 0.8148
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8847 - loss: 0.3066(3000, 500, 5)

Epoch 4 - MCC: 0.8256
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8847 - loss: 0.306

Training Model: LSTM, Fold: 5
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7292 - loss: 0.7446(3000, 500, 5)

Epoch 1 - MCC: 0.7650
750/750 ━━━━━━━━━━━━━━━━━━━━ 71s 94ms/step - accuracy: 0.7294 - loss: 0.7441 - val_accuracy: 0.8432 - val_loss: 0.4250 - mcc: 0.7650
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8577 - loss: 0.3827(3000, 500, 5)

Epoch 2 - MCC: 0.7965
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8577 - loss: 0.3827 - val_accuracy: 0.8671 - val_loss: 0.3496 - mcc: 0.7965
Epoch 3/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8710 - loss: 0.3423(3000, 500, 5)

Epoch 3 - MCC: 0.8031
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.8710 - loss: 0.3423 - val_accuracy: 0.8710 - val_loss: 0.3343 - mcc: 0.8031
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8802 - loss: 0.3162(3000, 500, 5)

Epoch 4 - MCC: 0.7555
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.8802 - loss: 0.316

...

Training Model: BiLSTM_Deep, Fold: 2
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8084 - loss: 0.5373(3000, 500, 5)

Epoch 1 - MCC: 0.8648
750/750 ━━━━━━━━━━━━━━━━━━━━ 56s 70ms/step - accuracy: 0.8084 - loss: 0.5371 - val_accuracy: 0.9121 - val_loss: 0.2311 - mcc: 0.8648
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9076 - loss: 0.2432(3000, 500, 5)

Epoch 2 - MCC: 0.8786
750/750 ━━━━━━━━━━━━━━━━━━━━ 79s 65ms/step - accuracy: 0.9076 - loss: 0.2431 - val_accuracy: 0.9209 - val_loss: 0.2022 - mcc: 0.8786
Epoch 3/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9182 - loss: 0.2086(3000, 500, 5)

Epoch 3 - MCC: 0.8838
750/750 ━━━━━━━━━━━━━━━━━━━━ 85s 69ms/step - accuracy: 0.9182 - loss: 0.2086 - val_accuracy: 0.9242 - val_loss: 0.1897 - mcc: 0.8838
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9261 - loss: 0.1851(3000, 500, 5)

Epoch 4 - MCC: 0.8948
750/750 ━━━━━━━━━━━━━━━━━━━━ 79s 66ms/step - accuracy: 0.9261 - loss

Training Model: BiLSTM_Deep, Fold: 3
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7989 - loss: 0.5556(3000, 500, 5)

Epoch 1 - MCC: 0.8500
750/750 ━━━━━━━━━━━━━━━━━━━━ 54s 68ms/step - accuracy: 0.7990 - loss: 0.5554 - val_accuracy: 0.9019 - val_loss: 0.2590 - mcc: 0.8500
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9075 - loss: 0.2459(3000, 500, 5)

Epoch 2 - MCC: 0.8741
750/750 ━━━━━━━━━━━━━━━━━━━━ 50s 66ms/step - accuracy: 0.9075 - loss: 0.2459 - val_accuracy: 0.9161 - val_loss: 0.2145 - mcc: 0.8741
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9186 - loss: 0.2104(3000, 500, 5)

Epoch 3 - MCC: 0.8783
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 66ms/step - accuracy: 0.9186 - loss: 0.2104 - val_accuracy: 0.9203 - val_loss: 0.2047 - mcc: 0.8783
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9242 - loss: 0.1923(3000, 500, 5)

Epoch 4 - MCC: 0.8828
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 66ms/step - accuracy: 0.9242 - loss

Training Model: BiLSTM_Deep, Fold: 4
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8126 - loss: 0.5186(3000, 500, 5)

Epoch 1 - MCC: 0.8502
750/750 ━━━━━━━━━━━━━━━━━━━━ 53s 66ms/step - accuracy: 0.8127 - loss: 0.5183 - val_accuracy: 0.9011 - val_loss: 0.2643 - mcc: 0.8502
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9070 - loss: 0.2490(3000, 500, 5)

Epoch 2 - MCC: 0.8601
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 68ms/step - accuracy: 0.9070 - loss: 0.2490 - val_accuracy: 0.9077 - val_loss: 0.2400 - mcc: 0.8601
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9167 - loss: 0.2160(3000, 500, 5)

Epoch 3 - MCC: 0.8891
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 69ms/step - accuracy: 0.9167 - loss: 0.2160 - val_accuracy: 0.9277 - val_loss: 0.1846 - mcc: 0.8891
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9270 - loss: 0.1832(3000, 500, 5)

Epoch 4 - MCC: 0.8948
750/750 ━━━━━━━━━━━━━━━━━━━━ 79s 66ms/step - accuracy: 0.9270 - loss

Training Model: BiLSTM_Deep, Fold: 5
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8009 - loss: 0.5612(3000, 500, 5)

Epoch 1 - MCC: 0.8615
750/750 ━━━━━━━━━━━━━━━━━━━━ 54s 68ms/step - accuracy: 0.8010 - loss: 0.5610 - val_accuracy: 0.9095 - val_loss: 0.2402 - mcc: 0.8615
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9144 - loss: 0.2246(3000, 500, 5)

Epoch 2 - MCC: 0.8735
750/750 ━━━━━━━━━━━━━━━━━━━━ 80s 65ms/step - accuracy: 0.9144 - loss: 0.2246 - val_accuracy: 0.9173 - val_loss: 0.2152 - mcc: 0.8735
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9233 - loss: 0.1980(3000, 500, 5)

Epoch 3 - MCC: 0.8838
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 64ms/step - accuracy: 0.9233 - loss: 0.1980 - val_accuracy: 0.9233 - val_loss: 0.1932 - mcc: 0.8838
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9288 - loss: 0.1803(3000, 500, 5)

Epoch 4 - MCC: 0.8933
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 66ms/step - accuracy: 0.9288 - loss

{'Accuracy': {'max': np.float64(0.9482513333333333),
              'mean': np.float64(0.9457213333333334),
              'min': np.float64(0.9441613333333333),
              'std': np.float64(0.0015069353152821307)},
 'Inference Time (s/sample)': {'max': np.float64(0.0018218497435251871),
                               'mean': np.float64(0.001107266060511271),
                               'min': np.float64(0.0008641552130381267),
                               'std': np.float64(0.0003589272276197002)},
 'MCC': {'max': np.float64(0.9210456857425257),
         'mean': np.float64(0.9173979803886871),
         'min': np.float64(0.9149237083816362),
         'std': np.float64(0.002258226938407201)},
 'Parameters': 88421,
 'Train Time (s)': {'max': np.float64(1480.8817269802094),
                    'mean': np.float64(1416.9027934551239),
                    'min': np.float64(1349.2553870677948),
                    'std': np.float64(47.11786699791029)},
 'Training Accuracy': [[0.863794922

Time estimation:

- 15min per model
- 6 models per set-up (1.5 hours per set-up)
- 2x4 different set-ups
- 15min x 6 x 2 x 4
- 720min = 12 Hours Total !!!!

Signal Only

In [33]:
basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/LSTM Models/New"

signal_multi_results, trained_models = train_and_evaluate(simple_models_dict, X=signal_data_vec, y=label_multi, epochs=n_epochs, dir_name="signal_multi")

filePath = f"{basePath}/20_Epoch_Signal_Multi_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(signal_multi_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5976 - loss: 1.1278(3000, 500, 5)

Epoch 1 - MCC: 0.5932
750/750 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.5980 - loss: 1.1267 - val_accuracy: 0.7387 - val_loss: 0.7401 - mcc: 0.5932
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7711 - loss: 0.6469(3000, 500, 5)

Epoch 2 - MCC: 0.6929
750/750 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.7711 - loss: 0.6469 - val_accuracy: 0.8014 - val_loss: 0.5464 - mcc: 0.6929
Epoch 3/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7975 - loss: 0.5719(3000, 500, 5)

Epoch 3 - MCC: 0.5360
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.7975 - loss: 0.5719 - val_accuracy: 0.7011 - val_loss: 0.8937 - mcc: 0.5360
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7769 - loss: 0.6506(3000, 500, 5)

Epoch 4 - MCC: 0.6868
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.7769 - loss: 0.650

Training Model: LSTM, Fold: 2
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5867 - loss: 1.1451(3000, 500, 5)

Epoch 1 - MCC: 0.6477
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.5869 - loss: 1.1445 - val_accuracy: 0.7772 - val_loss: 0.6524 - mcc: 0.6477
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7812 - loss: 0.6279(3000, 500, 5)

Epoch 2 - MCC: 0.6913
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.7812 - loss: 0.6279 - val_accuracy: 0.8031 - val_loss: 0.5452 - mcc: 0.6913
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8013 - loss: 0.5576(3000, 500, 5)

Epoch 3 - MCC: 0.6813
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.8013 - loss: 0.5576 - val_accuracy: 0.7970 - val_loss: 0.5888 - mcc: 0.6813
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7989 - loss: 0.5770(3000, 500, 5)

Epoch 4 - MCC: 0.6704
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.7989 - loss: 0.577

Training Model: LSTM, Fold: 3
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5896 - loss: 1.1485(3000, 500, 5)

Epoch 1 - MCC: 0.6553
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.5897 - loss: 1.1482 - val_accuracy: 0.7805 - val_loss: 0.6388 - mcc: 0.6553
Epoch 2/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7825 - loss: 0.6258(3000, 500, 5)

Epoch 2 - MCC: 0.6747
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.7825 - loss: 0.6258 - val_accuracy: 0.7921 - val_loss: 0.6107 - mcc: 0.6747
Epoch 3/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7962 - loss: 0.5919(3000, 500, 5)

Epoch 3 - MCC: 0.7070
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.7962 - loss: 0.5919 - val_accuracy: 0.8116 - val_loss: 0.5267 - mcc: 0.7070
Epoch 4/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8068 - loss: 0.5455(3000, 500, 5)

Epoch 4 - MCC: 0.6680
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.8068 - loss: 0.545

Training Model: LSTM, Fold: 4
Epoch 1/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5933 - loss: 1.1309(3000, 500, 5)

Epoch 1 - MCC: 0.6304
750/750 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.5936 - loss: 1.1300 - val_accuracy: 0.7664 - val_loss: 0.6793 - mcc: 0.6304
Epoch 2/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7837 - loss: 0.6123(3000, 500, 5)

Epoch 2 - MCC: 0.6949
750/750 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.7837 - loss: 0.6122 - val_accuracy: 0.8058 - val_loss: 0.5418 - mcc: 0.6949
Epoch 3/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7843 - loss: 0.6192(3000, 500, 5)

Epoch 3 - MCC: 0.6172
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.7843 - loss: 0.6192 - val_accuracy: 0.7578 - val_loss: 0.6837 - mcc: 0.6172
Epoch 4/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7443 - loss: 0.7271(3000, 500, 5)

Epoch 4 - MCC: 0.6620
750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 19ms/step - accuracy: 0.7444 - loss: 0.726

Training Model: LSTM, Fold: 5
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5957 - loss: 1.1444(3000, 500, 5)

Epoch 1 - MCC: 0.6453
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.5960 - loss: 1.1438 - val_accuracy: 0.7745 - val_loss: 0.6456 - mcc: 0.6453
Epoch 2/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7737 - loss: 0.6469(3000, 500, 5)

Epoch 2 - MCC: 0.6601
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.7737 - loss: 0.6469 - val_accuracy: 0.7829 - val_loss: 0.6223 - mcc: 0.6601
Epoch 3/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7929 - loss: 0.5945(3000, 500, 5)

Epoch 3 - MCC: 0.5642
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.7928 - loss: 0.5946 - val_accuracy: 0.7273 - val_loss: 0.8013 - mcc: 0.5642
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7560 - loss: 0.7057(3000, 500, 5)

Epoch 4 - MCC: 0.6849
750/750 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.7560 - loss: 0.705

...

Training Model: BiLSTM_Deep, Fold: 2
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.7260 - loss: 0.7711(3000, 500, 5)

Epoch 1 - MCC: 0.8565
750/750 ━━━━━━━━━━━━━━━━━━━━ 53s 66ms/step - accuracy: 0.7261 - loss: 0.7707 - val_accuracy: 0.9060 - val_loss: 0.2660 - mcc: 0.8565
Epoch 2/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8986 - loss: 0.2869(3000, 500, 5)

Epoch 2 - MCC: 0.8663
750/750 ━━━━━━━━━━━━━━━━━━━━ 49s 65ms/step - accuracy: 0.8986 - loss: 0.2869 - val_accuracy: 0.9127 - val_loss: 0.2347 - mcc: 0.8663
Epoch 3/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9125 - loss: 0.2379(3000, 500, 5)

Epoch 3 - MCC: 0.8784
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 65ms/step - accuracy: 0.9125 - loss: 0.2378 - val_accuracy: 0.9207 - val_loss: 0.2077 - mcc: 0.8784
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9185 - loss: 0.2182(3000, 500, 5)

Epoch 4 - MCC: 0.8905
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 64ms/step - accuracy: 0.9185 - loss

Training Model: BiLSTM_Deep, Fold: 3
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7416 - loss: 0.7381(3000, 500, 5)

Epoch 1 - MCC: 0.8508
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 65ms/step - accuracy: 0.7419 - loss: 0.7374 - val_accuracy: 0.9019 - val_loss: 0.2705 - mcc: 0.8508
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8994 - loss: 0.2817(3000, 500, 5)

Epoch 2 - MCC: 0.8623
750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 64ms/step - accuracy: 0.8994 - loss: 0.2817 - val_accuracy: 0.9098 - val_loss: 0.2455 - mcc: 0.8623
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9071 - loss: 0.2569(3000, 500, 5)

Epoch 3 - MCC: 0.8702
750/750 ━━━━━━━━━━━━━━━━━━━━ 47s 63ms/step - accuracy: 0.9071 - loss: 0.2569 - val_accuracy: 0.9150 - val_loss: 0.2325 - mcc: 0.8702
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9167 - loss: 0.2251(3000, 500, 5)

Epoch 4 - MCC: 0.8725
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 64ms/step - accuracy: 0.9167 - loss

Training Model: BiLSTM_Deep, Fold: 4
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7329 - loss: 0.7495(3000, 500, 5)

Epoch 1 - MCC: 0.8416
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 66ms/step - accuracy: 0.7331 - loss: 0.7492 - val_accuracy: 0.8971 - val_loss: 0.2992 - mcc: 0.8416
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9038 - loss: 0.2699(3000, 500, 5)

Epoch 2 - MCC: 0.8604
750/750 ━━━━━━━━━━━━━━━━━━━━ 80s 64ms/step - accuracy: 0.9038 - loss: 0.2699 - val_accuracy: 0.9086 - val_loss: 0.2405 - mcc: 0.8604
Epoch 3/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9160 - loss: 0.2241(3000, 500, 5)

Epoch 3 - MCC: 0.8793
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 65ms/step - accuracy: 0.9160 - loss: 0.2241 - val_accuracy: 0.9209 - val_loss: 0.2082 - mcc: 0.8793
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9209 - loss: 0.2099(3000, 500, 5)

Epoch 4 - MCC: 0.8750
750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 65ms/step - accuracy: 0.9209 - loss

Training Model: BiLSTM_Deep, Fold: 5
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7281 - loss: 0.7837(3000, 500, 5)

Epoch 1 - MCC: 0.8440
750/750 ━━━━━━━━━━━━━━━━━━━━ 55s 67ms/step - accuracy: 0.7282 - loss: 0.7833 - val_accuracy: 0.8982 - val_loss: 0.2948 - mcc: 0.8440
Epoch 2/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8993 - loss: 0.2857(3000, 500, 5)

Epoch 2 - MCC: 0.8548
750/750 ━━━━━━━━━━━━━━━━━━━━ 80s 64ms/step - accuracy: 0.8993 - loss: 0.2856 - val_accuracy: 0.9050 - val_loss: 0.2583 - mcc: 0.8548
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9042 - loss: 0.2662(3000, 500, 5)

Epoch 3 - MCC: 0.8193
750/750 ━━━━━━━━━━━━━━━━━━━━ 84s 67ms/step - accuracy: 0.9042 - loss: 0.2662 - val_accuracy: 0.8825 - val_loss: 0.3163 - mcc: 0.8193
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9143 - loss: 0.2296(3000, 500, 5)

Epoch 4 - MCC: 0.8794
750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 64ms/step - accuracy: 0.9143 - loss

{'Accuracy': {'max': np.float64(0.9425313333333334),
              'mean': np.float64(0.9374629333333333),
              'min': np.float64(0.9300713333333334),
              'std': np.float64(0.004630260095406216)},
 'Inference Time (s/sample)': {'max': np.float64(0.001075321594874064),
                               'mean': np.float64(0.0009070480187733968),
                               'min': np.float64(0.0008643518288930257),
                               'std': np.float64(8.413855745146769e-05)},
 'MCC': {'max': np.float64(0.9127377771092331),
         'mean': np.float64(0.9048789022330572),
         'min': np.float64(0.8938761371874021),
         'std': np.float64(0.00701588429050789)},
 'Parameters': 76133,
 'Train Time (s)': {'max': np.float64(1440.656200170517),
                    'mean': np.float64(1369.2872632026672),
                    'min': np.float64(1276.2334032058716),
                    'std': np.float64(56.59567585246842)},
 'Training Accuracy': [[0.820902585983

Hilbert Amps

In [31]:

basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/LSTM Models/New/"


hilbert_multi_results, trained_models = train_and_evaluate(simple_models_dict, X=hilbert_data_vec, y=label_multi, epochs=n_epochs, dir_name="hilbert_multi", basePath=basePath)


filePath = f"{basePath}/20_Epoch_Hilbert_Multi_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(hilbert_multi_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6891 - loss: 0.8477(3000, 500, 5)

Epoch 1 - MCC: 0.7294
750/750 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - accuracy: 0.6893 - loss: 0.8472 - val_accuracy: 0.8226 - val_loss: 0.4704 - mcc: 0.7294
Epoch 2/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8246 - loss: 0.4726(3000, 500, 5)

Epoch 2 - MCC: 0.7736
750/750 ━━━━━━━━━━━━━━━━━━━━ 18s 24ms/step - accuracy: 0.8246 - loss: 0.4725 - val_accuracy: 0.8528 - val_loss: 0.3845 - mcc: 0.7736
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8478 - loss: 0.4054(3000, 500, 5)

Epoch 3 - MCC: 0.7375
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.8478 - loss: 0.4054 - val_accuracy: 0.8301 - val_loss: 0.5058 - mcc: 0.7375
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8575 - loss: 0.3823(3000, 500, 5)

Epoch 4 - MCC: 0.7965
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8575 - loss: 0.382

Training Model: LSTM, Fold: 2
Epoch 1/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6604 - loss: 0.9228(3000, 500, 5)

Epoch 1 - MCC: 0.7020
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.6608 - loss: 0.9216 - val_accuracy: 0.8091 - val_loss: 0.5070 - mcc: 0.7020
Epoch 2/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8209 - loss: 0.4768(3000, 500, 5)

Epoch 2 - MCC: 0.7583
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.8209 - loss: 0.4767 - val_accuracy: 0.8436 - val_loss: 0.4085 - mcc: 0.7583
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8381 - loss: 0.4338(3000, 500, 5)

Epoch 3 - MCC: 0.7627
750/750 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step - accuracy: 0.8381 - loss: 0.4337 - val_accuracy: 0.8470 - val_loss: 0.4171 - mcc: 0.7627
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8513 - loss: 0.3974(3000, 500, 5)

Epoch 4 - MCC: 0.7805
750/750 ━━━━━━━━━━━━━━━━━━━━ 22s 20ms/step - accuracy: 0.8513 - loss: 0.397

Training Model: LSTM, Fold: 3
Epoch 1/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6965 - loss: 0.8270(3000, 500, 5)

Epoch 1 - MCC: 0.7257
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.6968 - loss: 0.8261 - val_accuracy: 0.8231 - val_loss: 0.4703 - mcc: 0.7257
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8343 - loss: 0.4387(3000, 500, 5)

Epoch 2 - MCC: 0.7703
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.8343 - loss: 0.4386 - val_accuracy: 0.8512 - val_loss: 0.3978 - mcc: 0.7703
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8490 - loss: 0.4028(3000, 500, 5)

Epoch 3 - MCC: 0.7597
750/750 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8490 - loss: 0.4028 - val_accuracy: 0.8448 - val_loss: 0.4404 - mcc: 0.7597
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8546 - loss: 0.3955(3000, 500, 5)

Epoch 4 - MCC: 0.7865
750/750 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.8546 - loss: 0.395

Training Model: LSTM, Fold: 4
Epoch 1/20
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6931 - loss: 0.8516(3000, 500, 5)

Epoch 1 - MCC: 0.7025
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.6934 - loss: 0.8509 - val_accuracy: 0.8085 - val_loss: 0.5248 - mcc: 0.7025
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8178 - loss: 0.4813(3000, 500, 5)

Epoch 2 - MCC: 0.7597
750/750 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - accuracy: 0.8178 - loss: 0.4813 - val_accuracy: 0.8448 - val_loss: 0.4099 - mcc: 0.7597
Epoch 3/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8467 - loss: 0.4083(3000, 500, 5)

Epoch 3 - MCC: 0.7736
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.8467 - loss: 0.4082 - val_accuracy: 0.8535 - val_loss: 0.3880 - mcc: 0.7736
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8586 - loss: 0.3752(3000, 500, 5)

Epoch 4 - MCC: 0.7784
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.8586 - loss: 0.375

Training Model: LSTM, Fold: 5
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6615 - loss: 0.9079(3000, 500, 5)

Epoch 1 - MCC: 0.6922
750/750 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.6616 - loss: 0.9077 - val_accuracy: 0.8009 - val_loss: 0.5449 - mcc: 0.6922
Epoch 2/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8156 - loss: 0.4993(3000, 500, 5)

Epoch 2 - MCC: 0.7520
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.8156 - loss: 0.4991 - val_accuracy: 0.8394 - val_loss: 0.4312 - mcc: 0.7520
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8481 - loss: 0.4087(3000, 500, 5)

Epoch 3 - MCC: 0.7811
750/750 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.8481 - loss: 0.4087 - val_accuracy: 0.8578 - val_loss: 0.3712 - mcc: 0.7811
Epoch 4/20
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8582 - loss: 0.3855(3000, 500, 5)

Epoch 4 - MCC: 0.7936
750/750 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.8583 - loss: 0.385

...

Training Model: BiLSTM_Deep, Fold: 2
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7812 - loss: 0.5860(3000, 500, 5)

Epoch 1 - MCC: 0.8414
750/750 ━━━━━━━━━━━━━━━━━━━━ 50s 62ms/step - accuracy: 0.7813 - loss: 0.5857 - val_accuracy: 0.8970 - val_loss: 0.2858 - mcc: 0.8414
Epoch 2/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9002 - loss: 0.2729(3000, 500, 5)

Epoch 2 - MCC: 0.8601
750/750 ━━━━━━━━━━━━━━━━━━━━ 45s 60ms/step - accuracy: 0.9002 - loss: 0.2729 - val_accuracy: 0.9091 - val_loss: 0.2546 - mcc: 0.8601
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9144 - loss: 0.2285(3000, 500, 5)

Epoch 3 - MCC: 0.8752
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 60ms/step - accuracy: 0.9144 - loss: 0.2285 - val_accuracy: 0.9187 - val_loss: 0.2124 - mcc: 0.8752
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9238 - loss: 0.1963(3000, 500, 5)

Epoch 4 - MCC: 0.8918
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.9238 - loss

Training Model: BiLSTM_Deep, Fold: 3
Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7761 - loss: 0.5908(3000, 500, 5)

Epoch 1 - MCC: 0.8582
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 64ms/step - accuracy: 0.7762 - loss: 0.5905 - val_accuracy: 0.9073 - val_loss: 0.2458 - mcc: 0.8582
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9062 - loss: 0.2505(3000, 500, 5)

Epoch 2 - MCC: 0.8742
750/750 ━━━━━━━━━━━━━━━━━━━━ 45s 60ms/step - accuracy: 0.9062 - loss: 0.2505 - val_accuracy: 0.9177 - val_loss: 0.2155 - mcc: 0.8742
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9198 - loss: 0.2079(3000, 500, 5)

Epoch 3 - MCC: 0.8818
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 60ms/step - accuracy: 0.9198 - loss: 0.2079 - val_accuracy: 0.9225 - val_loss: 0.1977 - mcc: 0.8818
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9259 - loss: 0.1883(3000, 500, 5)

Epoch 4 - MCC: 0.8818
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 59ms/step - accuracy: 0.9259 - loss

Training Model: BiLSTM_Deep, Fold: 4
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7594 - loss: 0.6512(3000, 500, 5)

Epoch 1 - MCC: 0.8300
750/750 ━━━━━━━━━━━━━━━━━━━━ 52s 65ms/step - accuracy: 0.7596 - loss: 0.6506 - val_accuracy: 0.8852 - val_loss: 0.3008 - mcc: 0.8300
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8973 - loss: 0.2773(3000, 500, 5)

Epoch 2 - MCC: 0.8780
750/750 ━━━━━━━━━━━━━━━━━━━━ 44s 59ms/step - accuracy: 0.8973 - loss: 0.2773 - val_accuracy: 0.9203 - val_loss: 0.2084 - mcc: 0.8780
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.9178 - loss: 0.2133(3000, 500, 5)

Epoch 3 - MCC: 0.8785
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 59ms/step - accuracy: 0.9178 - loss: 0.2133 - val_accuracy: 0.9208 - val_loss: 0.2005 - mcc: 0.8785
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9252 - loss: 0.1913(3000, 500, 5)

Epoch 4 - MCC: 0.8862
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 61ms/step - accuracy: 0.9252 - loss

Training Model: BiLSTM_Deep, Fold: 5
Epoch 1/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7577 - loss: 0.6572(3000, 500, 5)

Epoch 1 - MCC: 0.8298
750/750 ━━━━━━━━━━━━━━━━━━━━ 49s 61ms/step - accuracy: 0.7579 - loss: 0.6567 - val_accuracy: 0.8884 - val_loss: 0.3111 - mcc: 0.8298
Epoch 2/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9018 - loss: 0.2654(3000, 500, 5)

Epoch 2 - MCC: 0.8573
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 59ms/step - accuracy: 0.9018 - loss: 0.2654 - val_accuracy: 0.9058 - val_loss: 0.2406 - mcc: 0.8573
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8965 - loss: 0.2855(3000, 500, 5)

Epoch 3 - MCC: 0.8746
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 58ms/step - accuracy: 0.8965 - loss: 0.2854 - val_accuracy: 0.9177 - val_loss: 0.2139 - mcc: 0.8746
Epoch 4/20
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9150 - loss: 0.2248(3000, 500, 5)

Epoch 4 - MCC: 0.8688
750/750 ━━━━━━━━━━━━━━━━━━━━ 84s 61ms/step - accuracy: 0.9150 - loss

{'Accuracy': {'max': np.float64(0.9448586666666666),
              'mean': np.float64(0.9417373333333334),
              'min': np.float64(0.9390853333333333),
              'std': np.float64(0.001911754145048754)},
 'Inference Time (s/sample)': {'max': np.float64(0.001723623514175415),
                               'mean': np.float64(0.0010133616924285888),
                               'min': np.float64(0.000732048749923706),
                               'std': np.float64(0.00035915038545372355)},
 'MCC': {'max': np.float64(0.9160973739729593),
         'mean': np.float64(0.9112201282070316),
         'min': np.float64(0.90718377490913),
         'std': np.float64(0.0029584867615140504)},
 'Parameters': 78181,
 'Train Time (s)': {'max': np.float64(1575.5616037845612),
                    'mean': np.float64(1465.2052896499633),
                    'min': np.float64(1388.9571986198425),
                    'std': np.float64(63.04917060808317)},
 'Training Accuracy': [[0.84336423873

## Beta Binary Models

In [ ]:
data_vec_beta_trim = data_vec[:, 10:15, :]
label_vec_beta_trim = label_vec[:, 10:15, :]

In [ ]:
## Class distribution
unique, counts = np.unique(label_vec_beta_trim.flatten(), return_counts=True)
print(f"Class distribution: {dict(zip(unique, counts))}")
print(f"Noise  Percentage: {(counts[0]/(sum(counts))):.4f}")
print(f"Beta Percentage: {(counts[1]/(sum(counts))):.4f}")

Class distribution: {0: 394387, 3: 355613}
Noise  Percentage: 0.5258
Beta Percentage: 0.4742


In [ ]:
data_vec_shaped, label_vec_shaped = preprocess_data(data_vector=data_vec_beta_trim, label_vector=label_vec_beta_trim)

print(data_vec_shaped.shape)
print(label_vec_shaped.shape)

(100, 7500, 25)
(100, 7500, 1)


In [ ]:
new_seq_len = 500

sample_increase_factor = int(data_vec_shaped.shape[1] / new_seq_len)

data_vec_reshaped, label_vec_reshaped = split_sequences_keep_data(data_vec_shaped, label_vec_shaped, new_seq_len=new_seq_len)

print(data_vec_reshaped.shape)
print(label_vec_reshaped.shape)

(500, 1500, 25)
(500, 1500, 1)


In [ ]:
beta_hilbert_incl_index = [0,3]
beta_hilbert_only_index = [3]
beta_wavelet_incl_index = [0,12,13,14,15,16]
beta_wavelet_only_index = [12,13,14,15,16]
beta_allF_index         = beta_hilbert_incl_index + beta_wavelet_only_index

hilbert_data_vec = data_vec_reshaped[:,:,beta_hilbert_incl_index]
signal_data_vec = data_vec_reshaped[:,:, :1]
wavelet_data_vec = data_vec_reshaped[:,:, beta_wavelet_incl_index]
all_data_vec = data_vec_reshaped[:,:,beta_allF_index]

label_multi = label_vec_reshaped
label_binary = label_vec_reshaped.copy()
label_binary[label_binary >= 1] = 1  # Convert to binary labels

In [ ]:
print(signal_data_vec.shape)
print(hilbert_data_vec.shape)
print(wavelet_data_vec.shape)
print(all_data_vec.shape)
print(label_multi.shape)
print(label_binary.shape)

(500, 1500, 1)
(500, 1500, 2)
(500, 1500, 6)
(500, 1500, 7)
(500, 1500, 1)
(500, 1500, 1)


All Features Beta Binary

In [ ]:
allF_binary_results, trained_models = train_and_evaluate(simple_models_dict, X=all_data_vec, y=label_binary, epochs=n_epochs, dir_name="allF_binary")

basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/LSTM Models"

filePath = f"{basePath}/30_AllF_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(allF_binary_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5319 - loss: 0.6637
Epoch 1 - MCC: 0.6515
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.5358 - loss: 0.6614 - val_accuracy: 0.8115 - val_loss: 0.5591 - mcc: 0.6515
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8164 - loss: 0.5165
Epoch 2 - MCC: 0.7333
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.8166 - loss: 0.5147 - val_accuracy: 0.8670 - val_loss: 0.3588 - mcc: 0.7333
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8676 - loss: 0.3346
Epoch 3 - MCC: 0.7777
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8677 - loss: 0.3341 - val_accuracy: 0.8878 - val_loss: 0.2850 - mcc: 0.7777
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8844 - loss: 0.2849
Epoch 4 - MCC: 0.8001
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.8845 - loss: 0.2846 - val_accuracy: 0.8998 - val_loss: 0.2484 - mcc: 0.8001
Epoch 5/30
25/25 ━━━━━

Training Model: LSTM, Fold: 2
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5221 - loss: 0.6609
Epoch 1 - MCC: 0.5129
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.5233 - loss: 0.6599 - val_accuracy: 0.7282 - val_loss: 0.5836 - mcc: 0.5129
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7814 - loss: 0.5444
Epoch 2 - MCC: 0.6835
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.7826 - loss: 0.5426 - val_accuracy: 0.8406 - val_loss: 0.3997 - mcc: 0.6835
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8640 - loss: 0.3496
Epoch 3 - MCC: 0.7528
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8643 - loss: 0.3486 - val_accuracy: 0.8766 - val_loss: 0.3098 - mcc: 0.7528
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8889 - loss: 0.2795
Epoch 4 - MCC: 0.7795
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.8890 - loss: 0.2792 - val_accuracy: 0.8900 - val_loss: 0.2761 - mcc: 0.7795
Epoch 5/30
25/25 ━━━━━

Training Model: LSTM, Fold: 3
Epoch 1/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4954 - loss: 0.6730
Epoch 1 - MCC: 0.4517
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.4987 - loss: 0.6708 - val_accuracy: 0.6651 - val_loss: 0.5894 - mcc: 0.4517
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7427 - loss: 0.5474
Epoch 2 - MCC: 0.7134
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.7445 - loss: 0.5456 - val_accuracy: 0.8568 - val_loss: 0.3874 - mcc: 0.7134
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8612 - loss: 0.3571
Epoch 3 - MCC: 0.7558
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8615 - loss: 0.3559 - val_accuracy: 0.8781 - val_loss: 0.2932 - mcc: 0.7558
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8872 - loss: 0.2800
Epoch 4 - MCC: 0.8053
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8872 - loss: 0.2798 - val_accuracy: 0.9031 - val_loss: 0.2426 - mcc: 0.8053
Epoch 5/30
25/25 ━━━━━

Training Model: LSTM, Fold: 4
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5141 - loss: 0.6665
Epoch 1 - MCC: 0.5150
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.5153 - loss: 0.6655 - val_accuracy: 0.7057 - val_loss: 0.5770 - mcc: 0.5150
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7890 - loss: 0.5337
Epoch 2 - MCC: 0.7241
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.7899 - loss: 0.5321 - val_accuracy: 0.8623 - val_loss: 0.3728 - mcc: 0.7241
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8644 - loss: 0.3447
Epoch 3 - MCC: 0.7958
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8648 - loss: 0.3436 - val_accuracy: 0.8979 - val_loss: 0.2678 - mcc: 0.7958
Epoch 4/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.8940 - loss: 0.2652
Epoch 4 - MCC: 0.8141
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.8939 - loss: 0.2652 - val_accuracy: 0.9071 - val_loss: 0.2415 - mcc: 0.8141
Epoch 5/30
24/25 ━━━━━

Training Model: LSTM, Fold: 5
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.4975 - loss: 0.6663
Epoch 1 - MCC: 0.5892
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.4998 - loss: 0.6651 - val_accuracy: 0.7699 - val_loss: 0.5714 - mcc: 0.5892
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8160 - loss: 0.5239
Epoch 2 - MCC: 0.6957
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.8163 - loss: 0.5223 - val_accuracy: 0.8480 - val_loss: 0.3830 - mcc: 0.6957
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8648 - loss: 0.3415
Epoch 3 - MCC: 0.7494
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8651 - loss: 0.3407 - val_accuracy: 0.8750 - val_loss: 0.3025 - mcc: 0.7494
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8848 - loss: 0.2869
Epoch 4 - MCC: 0.7630
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8848 - loss: 0.2868 - val_accuracy: 0.8817 - val_loss: 0.2857 - mcc: 0.7630
Epoch 5/30
25/25 ━━━━━

...

Training Model: BiLSTM_Deep, Fold: 2
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.6607 - loss: 0.5930
Epoch 1 - MCC: 0.7311
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 241ms/step - accuracy: 0.6642 - loss: 0.5894 - val_accuracy: 0.8659 - val_loss: 0.3264 - mcc: 0.7311
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.8834 - loss: 0.2813
Epoch 2 - MCC: 0.8071
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.8839 - loss: 0.2802 - val_accuracy: 0.9034 - val_loss: 0.2294 - mcc: 0.8071
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9128 - loss: 0.2086
Epoch 3 - MCC: 0.8242
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 156ms/step - accuracy: 0.9132 - loss: 0.2078 - val_accuracy: 0.9110 - val_loss: 0.2179 - mcc: 0.8242
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.9237 - loss: 0.1864
Epoch 4 - MCC: 0.8421
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 164ms/step - accuracy: 0.9238 - loss: 0.1861 - val_accuracy: 0.9212 - val_loss: 0.1916 - mcc: 0.8421
Epoch 5

Training Model: BiLSTM_Deep, Fold: 3
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.5865 - loss: 0.6331
Epoch 1 - MCC: 0.7005
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 226ms/step - accuracy: 0.5905 - loss: 0.6304 - val_accuracy: 0.8504 - val_loss: 0.3999 - mcc: 0.7005
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.8656 - loss: 0.3454
Epoch 2 - MCC: 0.8265
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 166ms/step - accuracy: 0.8662 - loss: 0.3435 - val_accuracy: 0.9134 - val_loss: 0.2164 - mcc: 0.8265
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9182 - loss: 0.2020
Epoch 3 - MCC: 0.8421
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 178ms/step - accuracy: 0.9181 - loss: 0.2021 - val_accuracy: 0.9213 - val_loss: 0.1895 - mcc: 0.8421
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9254 - loss: 0.1823
Epoch 4 - MCC: 0.8538
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 0.9253 - loss: 0.1823 - val_accuracy: 0.9268 - val_loss: 0.1741 - mcc: 0.8538
Epoch 5

Training Model: BiLSTM_Deep, Fold: 4
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.6236 - loss: 0.5833
Epoch 1 - MCC: 0.7407
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 227ms/step - accuracy: 0.6282 - loss: 0.5798 - val_accuracy: 0.8681 - val_loss: 0.3260 - mcc: 0.7407
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.8634 - loss: 0.3217
Epoch 2 - MCC: 0.8135
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 196ms/step - accuracy: 0.8639 - loss: 0.3207 - val_accuracy: 0.9070 - val_loss: 0.2294 - mcc: 0.8135
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9070 - loss: 0.2267
Epoch 3 - MCC: 0.8517
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - accuracy: 0.9071 - loss: 0.2263 - val_accuracy: 0.9259 - val_loss: 0.1848 - mcc: 0.8517
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.9179 - loss: 0.1974
Epoch 4 - MCC: 0.8619
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 174ms/step - accuracy: 0.9181 - loss: 0.1969 - val_accuracy: 0.9311 - val_loss: 0.1670 - mcc: 0.8619
Epoch 5

Training Model: BiLSTM_Deep, Fold: 5
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.6441 - loss: 0.6130
Epoch 1 - MCC: 0.7308
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 243ms/step - accuracy: 0.6479 - loss: 0.6095 - val_accuracy: 0.8655 - val_loss: 0.3238 - mcc: 0.7308
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.8811 - loss: 0.2912
Epoch 2 - MCC: 0.7997
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - accuracy: 0.8814 - loss: 0.2905 - val_accuracy: 0.9000 - val_loss: 0.2479 - mcc: 0.7997
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9059 - loss: 0.2296
Epoch 3 - MCC: 0.8240
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 0.9061 - loss: 0.2293 - val_accuracy: 0.9115 - val_loss: 0.2145 - mcc: 0.8240
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9152 - loss: 0.2052
Epoch 4 - MCC: 0.8363
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 159ms/step - accuracy: 0.9154 - loss: 0.2048 - val_accuracy: 0.9183 - val_loss: 0.1955 - mcc: 0.8363
Epoch 

{'Accuracy': {'max': 0.95838,
              'mean': 0.9534866666666666,
              'min': 0.94322,
              'std': 0.005816187372803225},
 'Inference Time (s/sample)': {'max': 0.004210281372070313,
                               'mean': 0.0036422080993652344,
                               'min': 0.0032938408851623537,
                               'std': 0.0003049650722213964},
 'MCC': {'max': 0.9164583828547461,
         'mean': 0.9067859029470968,
         'min': 0.8865045557787847,
         'std': 0.01150249994389627},
 'Parameters': 79137,
 'Train Time (s)': {'max': 153.23102378845215,
                    'mean': 149.74760856628419,
                    'min': 147.28850412368774,
                    'std': 2.2713180443047642},
 'Training Accuracy': [[0.7286699414253235,
                        0.8924383521080017,
                        0.9125199317932129,
                        0.9221832156181335,
                        0.9275615811347961,
                        0.9288

Hilbert Beta Binary

In [ ]:
hilbert_binary_results, trained_models = train_and_evaluate(simple_models_dict, X=hilbert_data_vec, y=label_binary, epochs=n_epochs, dir_name="hilbert_binary")

basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/LSTM Models"

filePath = f"{basePath}/30_Hilbert_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(hilbert_binary_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5203 - loss: 0.6932
Epoch 1 - MCC: 0.3685
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.5217 - loss: 0.6924 - val_accuracy: 0.6268 - val_loss: 0.6248 - mcc: 0.3685
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6801 - loss: 0.5991
Epoch 2 - MCC: 0.6672
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.6823 - loss: 0.5978 - val_accuracy: 0.8339 - val_loss: 0.4623 - mcc: 0.6672
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8354 - loss: 0.4176
Epoch 3 - MCC: 0.7355
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8355 - loss: 0.4165 - val_accuracy: 0.8680 - val_loss: 0.3484 - mcc: 0.7355
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8573 - loss: 0.3505
Epoch 4 - MCC: 0.7505
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8574 - loss: 0.3501 - val_accuracy: 0.8757 - val_loss: 0.3071 - mcc: 0.7505
Epoch 5/30
25/25 ━━━━━

Training Model: LSTM, Fold: 2
Epoch 1/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5459 - loss: 0.6638
Epoch 1 - MCC: 0.4227
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.5480 - loss: 0.6621 - val_accuracy: 0.6624 - val_loss: 0.5978 - mcc: 0.4227
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7472 - loss: 0.5480
Epoch 2 - MCC: 0.6751
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.7490 - loss: 0.5462 - val_accuracy: 0.8377 - val_loss: 0.4004 - mcc: 0.6751
Epoch 3/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8479 - loss: 0.3730
Epoch 3 - MCC: 0.7173
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.8486 - loss: 0.3711 - val_accuracy: 0.8580 - val_loss: 0.3375 - mcc: 0.7173
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8766 - loss: 0.3015
Epoch 4 - MCC: 0.7725
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8768 - loss: 0.3011 - val_accuracy: 0.8865 - val_loss: 0.2823 - mcc: 0.7725
Epoch 5/30
25/25 ━━━━━

Training Model: LSTM, Fold: 3
Epoch 1/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5002 - loss: 0.6609
Epoch 1 - MCC: 0.4481
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.5023 - loss: 0.6595 - val_accuracy: 0.6651 - val_loss: 0.5907 - mcc: 0.4481
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7571 - loss: 0.5562
Epoch 2 - MCC: 0.7035
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.7587 - loss: 0.5547 - val_accuracy: 0.8511 - val_loss: 0.4010 - mcc: 0.7035
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8487 - loss: 0.3781
Epoch 3 - MCC: 0.7247
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.8490 - loss: 0.3771 - val_accuracy: 0.8612 - val_loss: 0.3291 - mcc: 0.7247
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8693 - loss: 0.3142
Epoch 4 - MCC: 0.7659
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8694 - loss: 0.3140 - val_accuracy: 0.8810 - val_loss: 0.2910 - mcc: 0.7659
Epoch 5/30
25/25 ━━━━━

Training Model: LSTM, Fold: 4
Epoch 1/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5036 - loss: 0.6743
Epoch 1 - MCC: 0.4128
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.5056 - loss: 0.6728 - val_accuracy: 0.6456 - val_loss: 0.6097 - mcc: 0.4128
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7193 - loss: 0.5796
Epoch 2 - MCC: 0.7059
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.7210 - loss: 0.5780 - val_accuracy: 0.8533 - val_loss: 0.4218 - mcc: 0.7059
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8481 - loss: 0.3920
Epoch 3 - MCC: 0.7420
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8481 - loss: 0.3913 - val_accuracy: 0.8713 - val_loss: 0.3290 - mcc: 0.7420
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8650 - loss: 0.3263
Epoch 4 - MCC: 0.7563
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8650 - loss: 0.3264 - val_accuracy: 0.8775 - val_loss: 0.2995 - mcc: 0.7563
Epoch 5/30
25/25 ━━━━━

Training Model: LSTM, Fold: 5
Epoch 1/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5403 - loss: 0.6904
Epoch 1 - MCC: 0.4421
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.5429 - loss: 0.6884 - val_accuracy: 0.6785 - val_loss: 0.6121 - mcc: 0.4421
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7367 - loss: 0.5749
Epoch 2 - MCC: 0.6754
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.7381 - loss: 0.5732 - val_accuracy: 0.8380 - val_loss: 0.4252 - mcc: 0.6754
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8432 - loss: 0.3873
Epoch 3 - MCC: 0.7101
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8435 - loss: 0.3864 - val_accuracy: 0.8547 - val_loss: 0.3446 - mcc: 0.7101
Epoch 4/30
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.8636 - loss: 0.3310
Epoch 4 - MCC: 0.7431
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.8640 - loss: 0.3302 - val_accuracy: 0.8717 - val_loss: 0.3122 - mcc: 0.7431
Epoch 5/30
24/25 ━━━━━

...

Training Model: BiLSTM_Deep, Fold: 2
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.6706 - loss: 0.5973
Epoch 1 - MCC: 0.6616
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 227ms/step - accuracy: 0.6736 - loss: 0.5939 - val_accuracy: 0.8296 - val_loss: 0.3831 - mcc: 0.6616
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.8456 - loss: 0.3442
Epoch 2 - MCC: 0.7678
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 176ms/step - accuracy: 0.8463 - loss: 0.3429 - val_accuracy: 0.8840 - val_loss: 0.2697 - mcc: 0.7678
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9019 - loss: 0.2315
Epoch 3 - MCC: 0.7897
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 158ms/step - accuracy: 0.9018 - loss: 0.2315 - val_accuracy: 0.8920 - val_loss: 0.2491 - mcc: 0.7897
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9118 - loss: 0.2059
Epoch 4 - MCC: 0.8224
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 181ms/step - accuracy: 0.9119 - loss: 0.2056 - val_accuracy: 0.9109 - val_loss: 0.2082 - mcc: 0.8224
Epoch 

Training Model: BiLSTM_Deep, Fold: 3
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.6478 - loss: 0.6184
Epoch 1 - MCC: 0.6700
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 256ms/step - accuracy: 0.6511 - loss: 0.6152 - val_accuracy: 0.8321 - val_loss: 0.3769 - mcc: 0.6700
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.8490 - loss: 0.3457
Epoch 2 - MCC: 0.7696
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 157ms/step - accuracy: 0.8492 - loss: 0.3452 - val_accuracy: 0.8852 - val_loss: 0.2750 - mcc: 0.7696
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.8866 - loss: 0.2682
Epoch 3 - MCC: 0.8194
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 205ms/step - accuracy: 0.8869 - loss: 0.2675 - val_accuracy: 0.9101 - val_loss: 0.2142 - mcc: 0.8194
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.9110 - loss: 0.2121
Epoch 4 - MCC: 0.8387
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step - accuracy: 0.9110 - loss: 0.2119 - val_accuracy: 0.9197 - val_loss: 0.1902 - mcc: 0.8387
Epoch 

Training Model: BiLSTM_Deep, Fold: 4
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.6202 - loss: 0.6177
Epoch 1 - MCC: 0.7110
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 262ms/step - accuracy: 0.6236 - loss: 0.6149 - val_accuracy: 0.8559 - val_loss: 0.3528 - mcc: 0.7110
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.8553 - loss: 0.3415
Epoch 2 - MCC: 0.7952
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 181ms/step - accuracy: 0.8556 - loss: 0.3405 - val_accuracy: 0.8967 - val_loss: 0.2494 - mcc: 0.7952
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.8995 - loss: 0.2376
Epoch 3 - MCC: 0.8519
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 174ms/step - accuracy: 0.8997 - loss: 0.2371 - val_accuracy: 0.9260 - val_loss: 0.1824 - mcc: 0.8519
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9168 - loss: 0.1983
Epoch 4 - MCC: 0.8635
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 158ms/step - accuracy: 0.9169 - loss: 0.1980 - val_accuracy: 0.9319 - val_loss: 0.1666 - mcc: 0.8635
Epoch 

Training Model: BiLSTM_Deep, Fold: 5
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.6124 - loss: 0.6132
Epoch 1 - MCC: 0.6408
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 264ms/step - accuracy: 0.6164 - loss: 0.6100 - val_accuracy: 0.8163 - val_loss: 0.4030 - mcc: 0.6408
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.8439 - loss: 0.3528
Epoch 2 - MCC: 0.7555
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.8445 - loss: 0.3517 - val_accuracy: 0.8749 - val_loss: 0.2896 - mcc: 0.7555
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.8986 - loss: 0.2417
Epoch 3 - MCC: 0.8153
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.8988 - loss: 0.2413 - val_accuracy: 0.9076 - val_loss: 0.2201 - mcc: 0.8153
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9191 - loss: 0.1934
Epoch 4 - MCC: 0.8248
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 0.9189 - loss: 0.1937 - val_accuracy: 0.9125 - val_loss: 0.2045 - mcc: 0.8248
Epoch 

{'Accuracy': {'max': 0.9581933333333333,
              'mean': 0.9508146666666667,
              'min': 0.9418466666666667,
              'std': 0.006158864830469967},
 'Inference Time (s/sample)': {'max': 0.00354550838470459,
                               'mean': 0.003401020050048828,
                               'min': 0.003199038505554199,
                               'std': 0.00016094781836579713},
 'MCC': {'max': 0.9161910083518204,
         'mean': 0.901506788087494,
         'min': 0.8836338629566973,
         'std': 0.012218529779988567},
 'Parameters': 76577,
 'Train Time (s)': {'max': 155.4189531803131,
                    'mean': 152.42602577209473,
                    'min': 149.31457614898682,
                    'std': 2.463563780602621},
 'Training Accuracy': [[0.7386633157730103,
                        0.8706033825874329,
                        0.9041282534599304,
                        0.9146900177001953,
                        0.9226967096328735,
            

Signal Beta Binary

In [ ]:
signal_binary_results, trained_models = train_and_evaluate(simple_models_dict, X=signal_data_vec, y=label_binary, epochs=n_epochs, dir_name="signal_binary")

basePath = "/content/drive/MyDrive/Colab Notebooks/Bachelor Thesis/Data/Model Comparisons/LSTM Models"

filePath = f"{basePath}/Signal_Binary_Model_Results.json"

with open(filePath, 'w') as f:
        json.dump(signal_binary_results, f, indent=4)  # indent=4 for pretty formatting

Training Model: LSTM, Fold: 1
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5527 - loss: 0.6897
Epoch 1 - MCC: 0.2208
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.5534 - loss: 0.6895 - val_accuracy: 0.6025 - val_loss: 0.6687 - mcc: 0.2208
Epoch 2/50
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.6288 - loss: 0.6541
Epoch 2 - MCC: 0.4983
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.6320 - loss: 0.6522 - val_accuracy: 0.7490 - val_loss: 0.5636 - mcc: 0.4983
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7548 - loss: 0.5328
Epoch 3 - MCC: 0.5841
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.7552 - loss: 0.5319 - val_accuracy: 0.7930 - val_loss: 0.4604 - mcc: 0.5841
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7921 - loss: 0.4605
Epoch 4 - MCC: 0.6401
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.7925 - loss: 0.4599 - val_accuracy: 0.8209 - val_loss: 0.4172 - mcc: 0.6401
Epoch 5/50
25/25 ━━━━━

Training Model: LSTM, Fold: 2
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5789 - loss: 0.6847
Epoch 1 - MCC: 0.2497
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.5796 - loss: 0.6844 - val_accuracy: 0.6130 - val_loss: 0.6604 - mcc: 0.2497
Epoch 2/50
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.6683 - loss: 0.6358
Epoch 2 - MCC: 0.4794
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.6714 - loss: 0.6337 - val_accuracy: 0.7354 - val_loss: 0.5651 - mcc: 0.4794
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7756 - loss: 0.5134
Epoch 3 - MCC: 0.5897
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.7759 - loss: 0.5125 - val_accuracy: 0.7954 - val_loss: 0.4630 - mcc: 0.5897
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8142 - loss: 0.4285
Epoch 4 - MCC: 0.6322
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8143 - loss: 0.4281 - val_accuracy: 0.8164 - val_loss: 0.4204 - mcc: 0.6322
Epoch 5/50
25/25 ━━━━━

Training Model: LSTM, Fold: 3
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5275 - loss: 0.6916
Epoch 1 - MCC: 0.2664
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.5291 - loss: 0.6914 - val_accuracy: 0.6283 - val_loss: 0.6677 - mcc: 0.2664
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6394 - loss: 0.6541
Epoch 2 - MCC: 0.4800
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.6404 - loss: 0.6534 - val_accuracy: 0.7388 - val_loss: 0.5741 - mcc: 0.4800
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7558 - loss: 0.5376
Epoch 3 - MCC: 0.6150
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.7564 - loss: 0.5363 - val_accuracy: 0.8081 - val_loss: 0.4534 - mcc: 0.6150
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8095 - loss: 0.4417
Epoch 4 - MCC: 0.6359
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8095 - loss: 0.4414 - val_accuracy: 0.8188 - val_loss: 0.4093 - mcc: 0.6359
Epoch 5/50
25/25 ━━━━━

Training Model: LSTM, Fold: 4
Epoch 1/50
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5625 - loss: 0.6864
Epoch 1 - MCC: 0.2432
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.5651 - loss: 0.6858 - val_accuracy: 0.6141 - val_loss: 0.6594 - mcc: 0.2432
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6436 - loss: 0.6413
Epoch 2 - MCC: 0.5156
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.6450 - loss: 0.6405 - val_accuracy: 0.7544 - val_loss: 0.5616 - mcc: 0.5156
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7635 - loss: 0.5298
Epoch 3 - MCC: 0.6203
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.7640 - loss: 0.5287 - val_accuracy: 0.8106 - val_loss: 0.4411 - mcc: 0.6203
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8138 - loss: 0.4340
Epoch 4 - MCC: 0.6634
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8138 - loss: 0.4338 - val_accuracy: 0.8322 - val_loss: 0.4015 - mcc: 0.6634
Epoch 5/50
25/25 ━━━━━

Training Model: LSTM, Fold: 5
Epoch 1/50
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5478 - loss: 0.6896
Epoch 1 - MCC: 0.1929
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.5489 - loss: 0.6892 - val_accuracy: 0.5878 - val_loss: 0.6673 - mcc: 0.1929
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6080 - loss: 0.6560
Epoch 2 - MCC: 0.4624
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.6096 - loss: 0.6553 - val_accuracy: 0.7279 - val_loss: 0.5802 - mcc: 0.4624
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7391 - loss: 0.5525
Epoch 3 - MCC: 0.5965
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.7400 - loss: 0.5510 - val_accuracy: 0.7987 - val_loss: 0.4473 - mcc: 0.5965
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8040 - loss: 0.4383
Epoch 4 - MCC: 0.6438
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8043 - loss: 0.4378 - val_accuracy: 0.8222 - val_loss: 0.4023 - mcc: 0.6438
Epoch 5/50
25/25 ━━━━━

...

Training Model: BiLSTM_Deep, Fold: 2
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.6789 - loss: 0.6486
Epoch 1 - MCC: 0.6678
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 219ms/step - accuracy: 0.6815 - loss: 0.6460 - val_accuracy: 0.8335 - val_loss: 0.3970 - mcc: 0.6678
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.8640 - loss: 0.3262
Epoch 2 - MCC: 0.7881
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 167ms/step - accuracy: 0.8644 - loss: 0.3251 - val_accuracy: 0.8939 - val_loss: 0.2531 - mcc: 0.7881
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.9110 - loss: 0.2123
Epoch 3 - MCC: 0.8302
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 207ms/step - accuracy: 0.9113 - loss: 0.2119 - val_accuracy: 0.9151 - val_loss: 0.2023 - mcc: 0.8302
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9268 - loss: 0.1775
Epoch 4 - MCC: 0.8354
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.9267 - loss: 0.1777 - val_accuracy: 0.9177 - val_loss: 0.1983 - mcc: 0.8354
Epoch 5

Training Model: BiLSTM_Deep, Fold: 3
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.6162 - loss: 0.6654
Epoch 1 - MCC: 0.6677
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 220ms/step - accuracy: 0.6187 - loss: 0.6636 - val_accuracy: 0.8346 - val_loss: 0.4583 - mcc: 0.6677
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.8470 - loss: 0.3969
Epoch 2 - MCC: 0.7925
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 190ms/step - accuracy: 0.8476 - loss: 0.3948 - val_accuracy: 0.8958 - val_loss: 0.2476 - mcc: 0.7925
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.8969 - loss: 0.2473
Epoch 3 - MCC: 0.8450
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.8972 - loss: 0.2465 - val_accuracy: 0.9222 - val_loss: 0.1895 - mcc: 0.8450
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.9223 - loss: 0.1885
Epoch 4 - MCC: 0.8533
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 191ms/step - accuracy: 0.9223 - loss: 0.1884 - val_accuracy: 0.9269 - val_loss: 0.1753 - mcc: 0.8533
Epoch 5

Training Model: BiLSTM_Deep, Fold: 4
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.6165 - loss: 0.6651
Epoch 1 - MCC: 0.6360
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 228ms/step - accuracy: 0.6192 - loss: 0.6631 - val_accuracy: 0.8185 - val_loss: 0.4415 - mcc: 0.6360
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.8299 - loss: 0.3906
Epoch 2 - MCC: 0.8223
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 176ms/step - accuracy: 0.8314 - loss: 0.3877 - val_accuracy: 0.9105 - val_loss: 0.2149 - mcc: 0.8223
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.9128 - loss: 0.2090
Epoch 3 - MCC: 0.8194
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 170ms/step - accuracy: 0.9129 - loss: 0.2086 - val_accuracy: 0.9087 - val_loss: 0.2131 - mcc: 0.8194
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9190 - loss: 0.1973
Epoch 4 - MCC: 0.8707
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 157ms/step - accuracy: 0.9191 - loss: 0.1971 - val_accuracy: 0.9352 - val_loss: 0.1631 - mcc: 0.8707
Epoch 

Training Model: BiLSTM_Deep, Fold: 5
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.6455 - loss: 0.6600
Epoch 1 - MCC: 0.6646
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 243ms/step - accuracy: 0.6490 - loss: 0.6580 - val_accuracy: 0.8317 - val_loss: 0.4312 - mcc: 0.6646
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.8502 - loss: 0.3678
Epoch 2 - MCC: 0.7868
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 202ms/step - accuracy: 0.8508 - loss: 0.3660 - val_accuracy: 0.8905 - val_loss: 0.2568 - mcc: 0.7868
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9073 - loss: 0.2255
Epoch 3 - MCC: 0.8324
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - accuracy: 0.9075 - loss: 0.2250 - val_accuracy: 0.9147 - val_loss: 0.2063 - mcc: 0.8324
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.9200 - loss: 0.1935
Epoch 4 - MCC: 0.8454
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 158ms/step - accuracy: 0.9201 - loss: 0.1932 - val_accuracy: 0.9228 - val_loss: 0.1869 - mcc: 0.8454
Epoch 

{'Accuracy': {'max': 0.9648466666666666,
              'mean': 0.9571626666666667,
              'min': 0.94808,
              'std': 0.006159279322922265},
 'Inference Time (s/sample)': {'max': 0.0035810399055480955,
                               'mean': 0.0035000085830688477,
                               'min': 0.0033470749855041503,
                               'std': 8.026211736623289e-05},
 'MCC': {'max': 0.9295314778327362,
         'mean': 0.914217540236238,
         'min': 0.8960745714453434,
         'std': 0.012217934326062305},
 'Parameters': 76065,
 'Train Time (s)': {'max': 255.8758499622345,
                    'mean': 247.70022249221802,
                    'min': 241.05864095687866,
                    'std': 5.697644946302527},
 'Training Accuracy': [[0.6716265678405762,
                        0.8692701458930969,
                        0.916606605052948,
                        0.9255232214927673,
                        0.9321717619895935,
                     

## k-fold with Noise Level Performance Data

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef, accuracy_score
from collections import defaultdict

# Assume we have a noise label array (same length as X)
# Example: noise_labels = np.random.randint(0, 3, len(X))  # Simulating 3 noise levels: 0, 1, 2
noise_labels = ...  # This should be an array of shape (len(X),)

# Create directory for model storage
os.makedirs("saved_models", exist_ok=True)

# Define model architectures
model_dict = {
    "Simple_LSTM": lambda input_shape, num_classes: tf.keras.Sequential([
        tf.keras.layers.LSTM(32, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]),
}

# Number of folds
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Initialize results storage
model_results = {model_name: {} for model_name in model_dict.keys()}
trained_models = {model_name: [] for model_name in model_dict.keys()}

# Convert y to a format compatible with StratifiedKFold
y_flat = np.argmax(y, axis=-1) if y.ndim == 3 else y  # Ensure shape (samples,)

for model_name, model_fn in model_dict.items():
    acc_scores, mcc_scores, train_times, infer_times = [], [], [], []
    noise_performance = defaultdict(lambda: {"acc": [], "mcc": []})  # Store per noise level

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_flat[:, 0])):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        val_noise_levels = noise_labels[val_idx]  # Get noise levels for validation set

        # Build and compile model
        model = model_fn(input_shape=X.shape[1:], num_classes=5)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Train model and measure time
        start_train = time.time()
        model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val), verbose=1)
        train_time = time.time() - start_train

        # Inference speed measurement
        start_infer = time.time()
        y_pred_probs = model.predict(X_val)
        infer_time = (time.time() - start_infer) / len(X_val)  # Time per sample

        # Compute metrics
        y_pred = np.argmax(y_pred_probs, axis=-1).flatten()
        y_true = y_val.flatten()

        acc = accuracy_score(y_true, y_pred)
        mcc = matthews_corrcoef(y_true, y_pred)

        acc_scores.append(acc)
        mcc_scores.append(mcc)
        train_times.append(train_time)
        infer_times.append(infer_time)

        # Save model for this fold
        model_save_path = f"saved_models/{model_name}_fold_{fold+1}.h5"
        model.save(model_save_path)
        trained_models[model_name].append(model_save_path)

        # **Noise-Level Performance Tracking**
        for noise_level in np.unique(val_noise_levels):
            mask = val_noise_levels == noise_level  # Get indices of this noise level
            y_true_noise = y_true[mask]
            y_pred_noise = y_pred[mask]

            if len(y_true_noise) > 0:  # Avoid empty cases
                acc_noise = accuracy_score(y_true_noise, y_pred_noise)
                mcc_noise = matthews_corrcoef(y_true_noise, y_pred_noise)

                noise_performance[noise_level]["acc"].append(acc_noise)
                noise_performance[noise_level]["mcc"].append(mcc_noise)

    # Store aggregated results
    model_results[model_name] = {
        "Accuracy": {"mean": np.mean(acc_scores), "std": np.std(acc_scores), "min": np.min(acc_scores), "max": np.max(acc_scores)},
        "MCC": {"mean": np.mean(mcc_scores), "std": np.std(mcc_scores), "min": np.min(mcc_scores), "max": np.max(mcc_scores)},
        "Train Time (s)": {"mean": np.mean(train_times), "std": np.std(train_times), "min": np.min(train_times), "max": np.max(train_times)},
        "Inference Time (s/sample)": {"mean": np.mean(infer_times), "std": np.std(infer_times), "min": np.min(infer_times), "max": np.max(infer_times)},
        "Noise Performance": {
            noise_level: {
                "Accuracy": {"mean": np.mean(scores["acc"]), "std": np.std(scores["acc"])},
                "MCC": {"mean": np.mean(scores["mcc"]), "std": np.std(scores["mcc"])}
            } for noise_level, scores in noise_performance.items()
        }
    }

    # Save the final trained model from the last fold
    final_model_save_path = f"saved_models/{model_name}_final.h5"
    model.save(final_model_save_path)

# Print results
import pprint
pprint.pprint(model_results)

# Save results as JSON
import json
with open("model_results.json", "w") as f:
    json.dump(model_results, f, indent=4)


## Visualize Metrics

In [ ]:
def plot_confusion_matrix(cm, class_names, title="Confusion Matrix"):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


In [ ]:
def plot_confusion_matrix(cm, class_names, title="Confusion Matrix"):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


In [ ]:
# Loss curve
def plot_loss_curves(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curves')
    plt.show()


Compuatational Efficiency

In [ ]:
def measure_inference_time(model, X_test):
    start_time = time.time()
    _ = model.predict(X_test)
    inference_time = time.time() - start_time
    return inference_time


In [ ]:
def measure_inference_time(model, X_test):
    start_time = time.time()
    _ = model.predict(X_test)
    inference_time = time.time() - start_time
    return inference_time


## Result Report

In [ ]:
results = {
    "Model": [],
    "Accuracy": [],
    "MAE": [],
    "F1-Score": [],
    "Training Time (s)": [],
    "Inference Time (s)": []
}

# Assuming `models` is a list of your models
for model_name, model in models.items():
    metrics = evaluate_model(model, X_train, y_train, X_val, y_val)
    inference_time = measure_inference_time(model, X_test)

    results["Model"].append(model_name)
    results["Accuracy"].append(metrics["accuracy"])
    results["MAE"].append(metrics["mae"])
    results["F1-Score"].append(metrics["f1"])
    results["Training Time (s)"].append(metrics["training_time"])
    results["Inference Time (s)"].append(inference_time)

# Convert to DataFrame for better visualization
results_df = pd.DataFrame(results)
print(results_df)
